# <center> **Определение уязвимых групп населения** 

Описание всех таблиц:

➔ child_mortality_rural_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в сельской местности.

➔ child_mortality_urban_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в городской местности.

➔ disabled_total_by_age_2017_2022.csv — число людей с инвалидностью по регионам, по месяцам, по возрастным группам.

➔ morbidity_2005_2020_age_disease.xls — заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний.

➔ poverty_percent_by_regions_1992_2020.csv — процент людей, живущих за чертой бедности (с денежными доходами ниже величины
прожиточного минимума), оценка за год по регионам.

➔ welfare_expense_share_2015_2020 — расходы на социальную политику от общих расходов бюджета региона, % в год*.

➔ cash_real_income_wages_2015_2020 — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная
зарплата, по регионам*.

➔ poverty_socdem_20*.xls — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся,
пенсионеры) за 2017–2020 гг., по регионам.

➔ housing_2020 — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого
помещения, обследование 2020 года*.

➔ population.xlsx — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг.

➔ gross_regional_product_1996_2020.xls — валовой региональный продукт на душу населения, в рублях.

<img src=titul_picture.jpg width=1500px height=450> 

Задача: 


* кластеризовать регионы России и определить, какие из них наиболее остро нуждаются в помощи малообеспеченным/неблагополучным слоям населения;
* описать группы населения, сталкивающиеся с бедностью;
* определить:
    - влияет ли число детей, пенсионеров и других социально уязвимых групп на уровень бедности в регионе;
    - связаны ли уровень бедности/социального неблагополучия с производством и потреблением в регионе;
    - какие ещё зависимости можно наблюдать относительно социально незащищённых слоёв населения.


## 1. Исследование данных

In [1]:
# Все необходимые библиотеки для работы.

import numpy as np
import pandas as pd
import re
import seaborn as sns
from matplotlib import pyplot as plt
import plotly 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline
plt.style.use('bmh')
from sklearn.cluster import KMeans

from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder,RobustScaler,StandardScaler

Нам необходимо создать полноценный датасет для дальнейшей работы, для этого мы в него будем добавлять всю нашу информацию постепенно.

Для удобства, запишем сразу словарь с разными названиями регионов. 

In [2]:
dct_city = {'Архангельская область':'Архангельская обл.(кроме НАО)',
'Архангельская область (кроме Ненецкого автономного округа)':'Архангельская обл.(кроме НАО)',             
'Город Москва столица Российской Федерации город федерального значения':'Москва',
'Город Санкт-Петербург город федерального значения':'Санкт-Петербург',
'Город федерального значения Севастополь':'Севастополь',
'Кемеровская область - Кузбасс':'Кемеровская область',
'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО.(Арх.обл.)',
'Ненецкий автономный округ':'Ненецкий АО.(Арх.обл.)',
'Республика Адыгея (Адыгея)':'Республика Адыгея',
'Республика Татарстан (Татарстан)':'Республика Татарстан',
'Тюменская область':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Ханты-Мансийский автономный округ - Югра (Тюменская область)':'ХМАО(Тюменская обл.)',
'Чувашская Республика - Чувашия':'Чувашская Республика',
'Ямало-Ненецкий автономный округ (Тюменская область)':'ЯНАО(Тюменская обл.)',
'Ямало-Ненецкий автономный округ':'ЯНАО(Тюменская обл.)',
'г. Москва':'Москва',
'г. Санкт-Петербург':'Санкт-Петербург',
'г. Севастополь':'Севастополь',
'город Москва':'Москва',
'город Санкт-Петербург':'Санкт-Петербург',
'город Севастополь':'Севастополь',
'Республика Северная Осетия - Алания':'Республика Северная Осетия-Алания',
'Республика Северная Осетия — Алания': 'Республика Северная Осетия-Алания',
'Ямало-Ненецкий  автономный округ':'ЯНАО(Тюменская обл.)',
'Ханты-Мансийский  автономный округ - Югра':'ХМАО(Тюменская обл.)',
'Ханты-Мансийский автономный округ — Югра':'ХМАО(Тюменская обл.)',
'Ханты-Мансийский автономный округ':'ХМАО(Тюменская обл.)',
'Архангельская область без автономного округа':'Архангельская обл.(кроме НАО)',
'Тюменская область без автономных округов':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Архангельская область без авт. округа':'Архангельская обл.(кроме НАО)',
'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Тюменская область без авт. округов':'Тюменская обл.(кроме ХМАО,ЯНАО)'}

### 1.1. Сбор данных

In [3]:
# выберем файл, где указаны все регионы.
data = pd.read_excel('./social_russia_data/poverty_socdem_2020.xls')

data = data.iloc[1:,0].reset_index(drop=True)
data = pd.DataFrame({'Регионы':data})
data['Регионы'] = data['Регионы'].apply(lambda x : x.strip())
data

,Регионы
0,Белгородская область
1,Брянская область
2,Владимирская область
3,Воронежская область
4,Ивановская область
...,...
82,Амурская область
83,Магаданская область
84,Сахалинская область
85,Еврейская автономная область


Удалим два региона, Тюменская и Архангельские области, т.к. они будут у нас расписаны как состав самой области и входящих в них автономных округов.

In [4]:
data.drop([20,61],inplace=True)
data = data.reset_index(drop=True)
data

,Регионы
0,Белгородская область
1,Брянская область
2,Владимирская область
3,Воронежская область
4,Ивановская область
...,...
80,Амурская область
81,Магаданская область
82,Сахалинская область
83,Еврейская автономная область


Итак у нас есть 85 регионов России. С ними мы будем работать.

In [5]:
# Приведем регионы к нашим стандартам.
data['Регионы'] = data['Регионы'].apply(lambda x :dct_city[x] if x in dct_city else x)

In [6]:
# 1. Численность населения 2020

# Прочитаем и запишем файл в переменную
data_population = pd.read_excel('./social_russia_data/population.xlsx')
data_population_2020 = data_population.iloc[2:,[0,23]]

data_population_2020.columns = ['region','Численность населения']

data_population_2020.iloc[:,1] = data_population_2020.iloc[:,1].shift(-1)
data_population_2020.dropna(inplace=True)
data_population_2020 = data_population_2020.reset_index(drop=True)

# Добавим в нашу таблицу столбец с наименованием федерального округа
mask_population = data_population_2020[(data_population_2020['region']).str.contains('федеральный')]

ind = mask_population.index
cols = list(mask_population['region'])


for num,value in enumerate(range(len(ind))):
    if num == len(mask_population.index)-1:
        v1 = ind[num]
        v2 = data_population_2020.index[-1] 
        data_population_2020.loc[v1:(v2),'Федеральный округ'] = cols[num]
    else:
        v1 = ind[num]
        v2 = ind[num+1]
        data_population_2020.loc[v1:(v2-1),'Федеральный округ'] = cols[num]

data_population_2020.head()

,region,Численность населения,Федеральный округ
0,Российская Федерация,146748590,NaN
1,Центральный федеральный округ,39433556,Центральный федеральный округ
2,Белгородская область,1549151,Центральный федеральный округ
3,Брянская область,1192491,Центральный федеральный округ
4,Владимирская область,1358416,Центральный федеральный округ


In [7]:
# Удалим строки с Тюменской и Архангельской областями
data_population_2020.drop([23,68],axis=0,inplace=True)

# Приведем названия регионов к общему виду, что бы данные подтянулись верно.
data_population_2020['region'] = data_population_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

In [8]:
# Подтянем данные к нашей таблице.
data = data.merge(right=data_population_2020,
           how='left',
           left_on='Регионы',
           right_on='region').drop(['region'],axis=1)

#  Отсортируем наши области по численности населения
data.sort_values(by='Численность населения',ascending=False,ignore_index=True,inplace=True)
data

,Регионы,Численность населения,Федеральный округ
0,Москва,12678079,Центральный федеральный округ
1,Московская область,7690863,Центральный федеральный округ
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016)
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ
4,Свердловская область,4310681,Уральский федеральный округ
...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ
81,Еврейская автономная область,158305,Дальневосточный федеральный округ
82,Магаданская область,140149,Дальневосточный федеральный округ
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ


In [9]:
# 2. новорождённые в этом месяце, по регионам, без учёта мертворождённых. Расчет взят на 15 июня каждого года.
data_newborn_2006_2022_monthly = pd.read_csv('./social_russia_data/newborn_2006_2022_monthly.csv',sep=';')
n = [0]

# Нам необходимо выделить 2020 год.
for num,value in enumerate(data_newborn_2006_2022_monthly.columns):
    lst = re.findall(r'\d{4}', value) 
    if len(lst) != 0: 
        if int(lst[0]) == 2020:
            n.append(num)
           
# Переименуем столбец        
data_newborn_2020 = data_newborn_2006_2022_monthly.iloc[:,n]
data_newborn_2020 = data_newborn_2020.rename(columns={'Region':'region'})

# Заменим запятые на точки
for i in data_newborn_2020.columns[1:]:
    data_newborn_2020[i] = data_newborn_2020[i].str.replace(',','.').astype('float')
mask = data_newborn_2020[data_newborn_2020.columns[1:]].sum(axis=1)

# Округлим данные    
data_newborn_2020 = pd.DataFrame({'region':data_newborn_2020['region'],'Новорожденные':np.round(mask)})

# Удалим строки с Тюменской и Архангельской областями, т.к. они ниже расписаны как области с АО и без него
data_newborn_2020.drop([23,71],axis=0,inplace=True)

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_newborn_2020['region'] = data_newborn_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

data_newborn_2020.head()


,region,Новорожденные
0,Российская Федерация,1435750.0
1,Центральный федеральный округ,354092.0
2,Белгородская область,12352.0
3,Брянская область,9480.0
4,Владимирская область,10120.0


In [10]:
# Присоединим данные
data = data.merge(right=data_newborn_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные
0,Москва,12678079,Центральный федеральный округ,123524.0
1,Московская область,7690863,Центральный федеральный округ,79107.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0
...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0


In [11]:
# 3.число умерших на первом году жизни детей за год, по всем регионам, в сельской местности. 

data_child_mortality_rural_1990_2021 = pd.read_excel('./social_russia_data/child_mortality_rural_1990_2021.xls')


data_child_mortality_rural_2020 = data_child_mortality_rural_1990_2021.iloc[2:,[0,33]]
data_child_mortality_rural_2020.columns = ['region','число умерших детей на первом году жизни в деревне 2020']
data_child_mortality_rural_2020['region'] = data_child_mortality_rural_2020['region'].apply(lambda x : x.strip())
data_child_mortality_rural_2020 = data_child_mortality_rural_2020.reset_index(drop=True)

# Удалим строки с Тюменской и Архангельской областями, т.к. они ниже расписаны как области с АО и без него
data_child_mortality_rural_2020.drop([23,71],axis=0,inplace=True)

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_child_mortality_rural_2020['region'] = data_child_mortality_rural_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

data_child_mortality_rural_2020.head()

,region,число умерших детей на первом году жизни в деревне 2020
0,Российская Федерация,1916.0
1,Центральный федеральный округ,228.0
2,Белгородская область,22.0
3,Брянская область,12.0
4,Владимирская область,10.0


In [12]:
# Присоединяем к основному датасету
data = data.merge(right=data_child_mortality_rural_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data 

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0
...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0


In [13]:
# 4. число умерших на первом году жизни детей за год, по всем регионам, в городской местности

data_child_mortality_urban_1990_2021 = pd.read_excel('./social_russia_data/child_mortality_urban_1990_2021.xls')
data_child_mortality_urban_2020 = data_child_mortality_urban_1990_2021.iloc[2:,[0,33]]
data_child_mortality_urban_2020.columns = ['region','число умерших детей на первом году жизни в городе 2020']
data_child_mortality_urban_2020['region'] = data_child_mortality_urban_2020['region'].apply(lambda x: x.strip())
data_child_mortality_urban_2020 = data_child_mortality_urban_2020.reset_index().drop(['index'],axis=1)

# Удалим строки с Тюменской и Архангельской областями, т.к. они ниже расписаны как области с АО и без него
data_child_mortality_urban_2020.drop([23,71],axis=0,inplace=True)

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_child_mortality_urban_2020['region'] = data_child_mortality_urban_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

data_child_mortality_urban_2020.head()

,region,число умерших детей на первом году жизни в городе 2020
0,Российская Федерация,4573.0
1,Центральный федеральный округ,1153.0
2,Белгородская область,25.0
3,Брянская область,29.0
4,Владимирская область,51.0


In [14]:
# Присоединяем к основному датасету
data = data.merge(right=data_child_mortality_urban_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0
...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0


In [15]:
# 5. число людей с инвалидностью по регионам, по месяцам, по возрастным группам.
data_disabled_total_by_age_2017_2022 = pd.read_csv('./social_russia_data/disabled_total_by_age_2017_2022.csv')
data_disabled_total_by_age_2017_2022['date'] = pd.to_datetime(data_disabled_total_by_age_2017_2022['date'],format='%Y-%m-%d')
dfc = data_disabled_total_by_age_2017_2022[data_disabled_total_by_age_2017_2022['date']=='2020-06-01']
data_disabled_total_by_age_2020 = dfc.groupby('region')[['total','18_30','31_40','41_50','51_60','60_']].sum().reset_index()
data_disabled_total_by_age_2020.columns = ['region','всего инвалидов','18_30_инвалид','31_40_инвалид','41_50_инвалид','51_60_инвалид','60_инвалид']

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_disabled_total_by_age_2020['region'] = data_disabled_total_by_age_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 

In [16]:
# Присоединяем к основному датасету
data = data.merge(right=data_disabled_total_by_age_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,135177.0,718314.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,65133.0,269936.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,73035.0,268746.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,56731.0,443593.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,40626.0,167167.0
...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,3405.0,13053.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,1782.0,7462.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,946.0,2499.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,395.0,563.0


In [17]:
# 6. процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам

data_poverty_percent_by_regions_1992_2020 = pd.read_csv('./social_russia_data/poverty_percent_by_regions_1992_2020.csv')
data_poverty_percent_by_regions_1992_2020[data_poverty_percent_by_regions_1992_2020['year']==2020].iloc[:,[0,2]]
data_poverty_percent_by_regions_2020 = data_poverty_percent_by_regions_1992_2020[
    data_poverty_percent_by_regions_1992_2020['year']==2020].iloc[:,[0,2]]

data_poverty_percent_by_regions_2020 = data_poverty_percent_by_regions_2020.rename(columns={'poverty_percent':'за чертой бедности в %'})

# Уберем пробелы по краям строк
data_poverty_percent_by_regions_2020['region'] = data_poverty_percent_by_regions_2020['region'].apply(lambda x: x.strip())

# Удалим строки с Тюменской и Архангельской областями, т.к. они ниже расписаны как области с АО и без него
data_poverty_percent_by_regions_2020.drop([606,1672],axis=0,inplace=True)

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_poverty_percent_by_regions_2020['region'] = data_poverty_percent_by_regions_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 

data_poverty_percent_by_regions_2020.head()

,region,за чертой бедности в %
28,Российская Федерация,12.1
70,Белгородская область,7.2
96,Брянская область,13.6
122,Владимирская область,12.5
148,Воронежская область,8.5


In [18]:
# Присоединяем к основному датасету
data = data.merge(right=data_poverty_percent_by_regions_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)

data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид,за чертой бедности в %
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,135177.0,718314.0,6.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,65133.0,269936.0,6.8
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,73035.0,268746.0,10.4
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,56731.0,443593.0,5.9
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,40626.0,167167.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,3405.0,13053.0,23.8
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,1782.0,7462.0,23.6
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,946.0,2499.0,8.7
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,395.0,563.0,8.0


In [19]:
# 7. распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам.
data_poverty_socdem_2021 = pd.read_excel('./social_russia_data/poverty_socdem_2021.xls',header=2,usecols=[0,2,3,4])
data_poverty_socdem_2021.columns=['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']

# удаляем пропуски по краям в названиях регионов 
data_poverty_socdem_2021['region'] = data_poverty_socdem_2021['region'].apply(lambda x: x.strip())

data_poverty_socdem_2021['region'] = data_poverty_socdem_2021['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_poverty_socdem_2021.head()

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
0,Российская Федерация,40.7,5.5,53.8
1,Центральный федеральный округ,0.0,0.0,0.0
2,Белгородская область,43.0,12.9,44.0
3,Брянская область,45.0,3.6,51.4
4,Владимирская область,36.4,4.9,58.7


In [20]:
# Добавляем данные в наш датасет.

data = data.merge(right=data_poverty_socdem_2021,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид,за чертой бедности в %,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,135177.0,718314.0,6.0,28.7,15.0,56.3
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,65133.0,269936.0,6.8,43.4,6.6,50.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,73035.0,268746.0,10.4,43.4,6.7,49.9
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,56731.0,443593.0,5.9,36.6,4.2,59.2
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,40626.0,167167.0,9.0,46.0,3.6,50.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,3405.0,13053.0,23.8,42.9,1.8,55.3
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,1782.0,7462.0,23.6,36.5,7.8,55.7
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,946.0,2499.0,8.7,39.1,2.6,58.3
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,395.0,563.0,8.0,NaN,NaN,NaN


In [21]:
# 8. болезни
data_morbidity_2020_2021_age_disease = pd.read_excel('./social_russia_data/morbidity_2005_2020_age_disease.xls',
                                                     header=2,usecols=[0,1,2,3],
                                                     names=['region','вид болезни','возраст',2020,2021])

# удаляем пропуски по краям в названиях регионов 
data_morbidity_2020_2021_age_disease['region'] = data_morbidity_2020_2021_age_disease['region'].apply(lambda x: x.strip())

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_morbidity_2020_2021_age_disease['region'] = data_morbidity_2020_2021_age_disease['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 

# Убираем строки, если значения в столбцах вид болезни равно значению Все заболевания и в столбце Возраст Всего
data_morbidity_2020_2021_age_disease = data_morbidity_2020_2021_age_disease[(data_morbidity_2020_2021_age_disease['вид болезни'] !='Все заболевания')&(data_morbidity_2020_2021_age_disease['возраст'] !='Всего') ]

# складываем регионы и виды болезней,а остальные значения суммируем
data_morbidity_2020_2021_age_disease  = data_morbidity_2020_2021_age_disease.groupby(by =['region','вид болезни'],as_index=False).sum().round()
data_morbidity_2020 = data_morbidity_2020_2021_age_disease.pivot_table(columns='вид болезни',index='region',values=2020).reset_index()


data_morbidity_2020.head()

вид болезни,region,"Беременность, роды и послеродовой период",Болезни глаза и его придаточного аппарата,Болезни кожи и подкожной клетчатки,Болезни костно-мышечной системы и соединительной ткани,"Болезни крови, кроветворных органов и отдельные нарушения, вовлекающие иммунный механизм",Болезни мочеполовой системы,Болезни нервной системы,"Болезни нервной системы, болезни глаза и его придаточного аппарата, болезни уха и сосцевидного отростка",Болезни органов дыхания,...,Болезни системы кровообращения,Болезни уха и сосцевидного отростка,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Врожденные аномалии (пороки развития), деформации и хромосомные нарушения",Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин"
0,Алтайский край,10551.0,10070.0,14842.0,11309.0,1732.0,20306.0,5923.0,8131.0,243932.0,...,6834.0,6640.0,5426.0,981.0,7214.0,2979.0,659.0,3003.0,407.0,25545.0
1,Амурская область,4426.0,13860.0,15952.0,9032.0,1794.0,11059.0,4733.0,7418.0,209505.0,...,4667.0,10271.0,5094.0,1274.0,9099.0,1585.0,2560.0,1055.0,0.0,30574.0
2,Архангельская обл.(кроме НАО),8100.0,13396.0,21709.0,16367.0,2360.0,15316.0,6083.0,7992.0,255461.0,...,6802.0,10409.0,8564.0,1999.0,16704.0,3509.0,2732.0,2249.0,0.0,43801.0
3,Астраханская область,6271.0,11103.0,5483.0,7372.0,1107.0,9544.0,4330.0,6648.0,155093.0,...,4621.0,9619.0,7111.0,1966.0,6383.0,1367.0,2738.0,1266.0,0.0,16789.0
4,Белгородская область,4167.0,8897.0,13960.0,7890.0,908.0,13224.0,7061.0,10001.0,176037.0,...,5698.0,10052.0,3021.0,904.0,12796.0,1620.0,1442.0,1269.0,8.0,31244.0


In [22]:
# Добавляем данные в наш датасет.
data = data.merge(right=data_morbidity_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Болезни системы кровообращения,Болезни уха и сосцевидного отростка,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Врожденные аномалии (пороки развития), деформации и хромосомные нарушения",Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин"
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,2770.0,9026.0,3482.0,769.0,8440.0,2631.0,270.0,1215.0,0.0,34768.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,4544.0,8298.0,2453.0,521.0,8763.0,1983.0,907.0,1384.0,0.0,36166.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,8448.0,7310.0,4456.0,795.0,6248.0,1796.0,919.0,1083.0,1085.0,31054.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,6798.0,12750.0,6938.0,1845.0,15860.0,4348.0,1399.0,1922.0,0.0,40689.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,5346.0,8748.0,6037.0,614.0,9931.0,1594.0,2358.0,2096.0,781.0,28541.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,4580.0,9046.0,5557.0,923.0,14954.0,2064.0,1455.0,515.0,714.0,20697.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,4634.0,4246.0,1652.0,1121.0,9180.0,1595.0,2098.0,644.0,7.0,18832.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,2438.0,5618.0,4140.0,1270.0,6592.0,1614.0,1687.0,1222.0,1401.0,37154.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,4600.0,9932.0,2234.0,999.0,15248.0,3387.0,114.0,3104.0,2502.0,35558.0


In [23]:
# 9.расходы на социальную политику от общих расходов бюджета региона, % в год*

data_welfare_expense_share_2020 = pd.read_excel('./social_russia_data/welfare_expense_share_2015_2020.xlsx',
                                                     usecols=[0,6],
                                                     names=['region','расходы на соц.пол. от бюджета %'])

# Убираем пробелы и переносы
data_welfare_expense_share_2020['region'] = data_welfare_expense_share_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

data_welfare_expense_share_2020['region'] = data_welfare_expense_share_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_welfare_expense_share_2020['region'] = data_welfare_expense_share_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 

data_welfare_expense_share_2020.head()

,region,расходы на соц.пол. от бюджета %
0,Российская Федерация,21.3
1,Центральный федеральный округ,NaN
2,Белгородская область,16.1
3,Брянская область,23.1
4,Владимирская область,26.0


In [24]:
# Добавляем данные в наш датасет.
data = data.merge(right=data_welfare_expense_share_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Болезни уха и сосцевидного отростка,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Врожденные аномалии (пороки развития), деформации и хромосомные нарушения",Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин",расходы на соц.пол. от бюджета %
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,9026.0,3482.0,769.0,8440.0,2631.0,270.0,1215.0,0.0,34768.0,17.5
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,8298.0,2453.0,521.0,8763.0,1983.0,907.0,1384.0,0.0,36166.0,19.4
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,7310.0,4456.0,795.0,6248.0,1796.0,919.0,1083.0,1085.0,31054.0,25.7
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,12750.0,6938.0,1845.0,15860.0,4348.0,1399.0,1922.0,0.0,40689.0,19.8
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,8748.0,6037.0,614.0,9931.0,1594.0,2358.0,2096.0,781.0,28541.0,23.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,9046.0,5557.0,923.0,14954.0,2064.0,1455.0,515.0,714.0,20697.0,19.8
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,4246.0,1652.0,1121.0,9180.0,1595.0,2098.0,644.0,7.0,18832.0,31.2
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,5618.0,4140.0,1270.0,6592.0,1614.0,1687.0,1222.0,1401.0,37154.0,11.1
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,9932.0,2234.0,999.0,15248.0,3387.0,114.0,3104.0,2502.0,35558.0,6.4


In [29]:
# 10.среднедушевые денежные доходы населения
per_capita_cash_income_2019_2020 = pd.read_excel(
                io='./social_russia_data/cash_real_income_wages_2015_2020.xlsx',
                usecols=[0,6],
                names=['region','Среднедушевые доходы 2020'],
                sheet_name=0)

# Убираем пробелы и переносы
per_capita_cash_income_2019_2020['region'] = per_capita_cash_income_2019_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

# Удаляем строки, которые касаются сумарным значениям Тюменской и Архангельской областей 
per_capita_cash_income_2019_2020 = per_capita_cash_income_2019_2020.drop([23,69]).reset_index(drop=True)

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
per_capita_cash_income_2019_2020['region'] = per_capita_cash_income_2019_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

per_capita_cash_income_2019_2020.head()

,region,Среднедушевые доходы 2020
0,Российская Федерация,36073.0
1,Центральный федеральный округ,48226.0
2,Белгородская область,32841.0
3,Брянская область,28596.0
4,Владимирская область,25922.0


In [30]:
# Добавляем данные в наш датасет.
data = data.merge(right=per_capita_cash_income_2019_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,"Болезни эндокринной системы, расстройства питания и нарушения обмена веществ","Врожденные аномалии (пороки развития), деформации и хромосомные нарушения",Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин",расходы на соц.пол. от бюджета %,Среднедушевые доходы 2020
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,3482.0,769.0,8440.0,2631.0,270.0,1215.0,0.0,34768.0,17.5,77283.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,2453.0,521.0,8763.0,1983.0,907.0,1384.0,0.0,36166.0,19.4,47046.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,4456.0,795.0,6248.0,1796.0,919.0,1083.0,1085.0,31054.0,25.7,36838.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,6938.0,1845.0,15860.0,4348.0,1399.0,1922.0,0.0,40689.0,19.8,49207.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,6037.0,614.0,9931.0,1594.0,2358.0,2096.0,781.0,28541.0,23.5,37374.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,5557.0,923.0,14954.0,2064.0,1455.0,515.0,714.0,20697.0,19.8,21677.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,1652.0,1121.0,9180.0,1595.0,2098.0,644.0,7.0,18832.0,31.2,28048.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,4140.0,1270.0,6592.0,1614.0,1687.0,1222.0,1401.0,37154.0,11.1,70864.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,2234.0,999.0,15248.0,3387.0,114.0,3104.0,2502.0,35558.0,6.4,89541.0


In [31]:
# 11.реальные денежные доходы населения
Real_income_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',sheet_name=1,usecols=[0,6])
Real_income_2020 = Real_income_2020.rename(columns={2020:'реальные доходы'}) 

# Удаляем строки, которые касаются сумарным значениям Тюменской и Архангельской областей 
Real_income_2020 = Real_income_2020.drop([23,69]).reset_index(drop=True)

# Убираем пробелы и переносы
Real_income_2020['region'] = Real_income_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
Real_income_2020['region'] = Real_income_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)


Real_income_2020.head()

,region,реальные доходы
0,Российская Федерация,98.6
1,Центральный федеральный округ,98.7
2,Белгородская область,98.1
3,Брянская область,96.3
4,Владимирская область,98.0


In [32]:
# Добавляем данные в наш датасет.
data = data.merge(right=Real_income_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,"Врожденные аномалии (пороки развития), деформации и хромосомные нарушения",Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин",расходы на соц.пол. от бюджета %,Среднедушевые доходы 2020,реальные доходы
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,769.0,8440.0,2631.0,270.0,1215.0,0.0,34768.0,17.5,77283.0,100.2
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,521.0,8763.0,1983.0,907.0,1384.0,0.0,36166.0,19.4,47046.0,97.8
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,795.0,6248.0,1796.0,919.0,1083.0,1085.0,31054.0,25.7,36838.0,100.4
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,1845.0,15860.0,4348.0,1399.0,1922.0,0.0,40689.0,19.8,49207.0,101.1
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,614.0,9931.0,1594.0,2358.0,2096.0,781.0,28541.0,23.5,37374.0,92.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,923.0,14954.0,2064.0,1455.0,515.0,714.0,20697.0,19.8,21677.0,104.5
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,1121.0,9180.0,1595.0,2098.0,644.0,7.0,18832.0,31.2,28048.0,99.2
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,1270.0,6592.0,1614.0,1687.0,1222.0,1401.0,37154.0,11.1,70864.0,104.3
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,999.0,15248.0,3387.0,114.0,3104.0,2502.0,35558.0,6.4,89541.0,105.2


In [35]:
# 12.номинальная начисленная зарплата, по регионам.
Nominal_accrued_salary_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',
                                            sheet_name=2,
                                            usecols=[0,6],
                                            names=['region','номинальная зарплата 2020'])

# Удаляем строки, которые касаются сумарным значениям Тюменской и Архангельской областей 
Nominal_accrued_salary_2020 = Nominal_accrued_salary_2020.drop([23,69]).reset_index(drop=True)

# Убираем пробелы и переносы
Nominal_accrued_salary_2020['region'] = Nominal_accrued_salary_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
Nominal_accrued_salary_2020['region'] = Nominal_accrued_salary_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)


Nominal_accrued_salary_2020


,region,номинальная зарплата 2020
0,Российская Федерация,51344.0
1,Центральный федеральный округ,65319.0
2,Белгородская область,37442.0
3,Брянская область,31946.0
4,Владимирская область,35240.0
...,...,...
91,Амурская область,52430.0
92,Магаданская область,102843.0
93,Сахалинская область,92518.0
94,Еврейская автономная область,46237.0


In [36]:
# Добавляем данные в наш датасет.
data = data.merge(right=Nominal_accrued_salary_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Некоторые инфекционные и паразитарные болезни,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин",расходы на соц.пол. от бюджета %,Среднедушевые доходы 2020,реальные доходы,номинальная зарплата 2020
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,8440.0,2631.0,270.0,1215.0,0.0,34768.0,17.5,77283.0,100.2,100070.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,8763.0,1983.0,907.0,1384.0,0.0,36166.0,19.4,47046.0,97.8,58066.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,6248.0,1796.0,919.0,1083.0,1085.0,31054.0,25.7,36838.0,100.4,38499.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,15860.0,4348.0,1399.0,1922.0,0.0,40689.0,19.8,49207.0,101.1,68667.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,9931.0,1594.0,2358.0,2096.0,781.0,28541.0,23.5,37374.0,92.5,43256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,14954.0,2064.0,1455.0,515.0,714.0,20697.0,19.8,21677.0,104.5,36269.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,9180.0,1595.0,2098.0,644.0,7.0,18832.0,31.2,28048.0,99.2,46237.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,6592.0,1614.0,1687.0,1222.0,1401.0,37154.0,11.1,70864.0,104.3,102843.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,15248.0,3387.0,114.0,3104.0,2502.0,35558.0,6.4,89541.0,105.2,120641.0


In [37]:
# 13.Реальная начисленная зарплата по регионам
Real_accrued_salary_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',
                                         sheet_name=3,
                                         usecols=[0,6],
                                         names=['region','начисленная зп. реал. 2020'])

# Удаляем строки, которые касаются сумарным значениям Тюменской и Архангельской областей 
Real_accrued_salary_2020 = Real_accrued_salary_2020.drop([23,69]).reset_index(drop=True)

# Убираем пробелы и переносы
Real_accrued_salary_2020['region'] = Real_accrued_salary_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
Real_accrued_salary_2020['region'] = Real_accrued_salary_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

Real_accrued_salary_2020

,region,начисленная зп. реал. 2020
0,Российская Федерация,103.8
1,Центральный федеральный округ,104.2
2,Белгородская область,104.8
3,Брянская область,102.9
4,Владимирская область,103.0
...,...,...
91,Амурская область,105.2
92,Магаданская область,105.0
93,Сахалинская область,101.7
94,Еврейская автономная область,103.7


In [38]:
# Добавляем данные в наш датасет.
data = data.merge(right=Real_accrued_salary_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Новообразования,"Отдельные состояния, возникающие в перинатальном периоде",Психические расстройства и расстройства поведения,"Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях","Травмы, отравления и другие последствия воздействия внешних причин",расходы на соц.пол. от бюджета %,Среднедушевые доходы 2020,реальные доходы,номинальная зарплата 2020,начисленная зп. реал. 2020
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,2631.0,270.0,1215.0,0.0,34768.0,17.5,77283.0,100.2,100070.0,103.1
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,1983.0,907.0,1384.0,0.0,36166.0,19.4,47046.0,97.8,58066.0,101.8
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,1796.0,919.0,1083.0,1085.0,31054.0,25.7,36838.0,100.4,38499.0,103.2
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,4348.0,1399.0,1922.0,0.0,40689.0,19.8,49207.0,101.1,68667.0,101.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,1594.0,2358.0,2096.0,781.0,28541.0,23.5,37374.0,92.5,43256.0,102.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,2064.0,1455.0,515.0,714.0,20697.0,19.8,21677.0,104.5,36269.0,105.5
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,1595.0,2098.0,644.0,7.0,18832.0,31.2,28048.0,99.2,46237.0,103.7
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,1614.0,1687.0,1222.0,1401.0,37154.0,11.1,70864.0,104.3,102843.0,105.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,3387.0,114.0,3104.0,2502.0,35558.0,6.4,89541.0,105.2,120641.0,110.5


In [39]:
# Для дальнейшей работы нам понадобится добавить в наш словарь регионов новые названия
dct_city.update({
'Bладимирская область':'Владимирская область',
'Bолгоградская область':'Волгоградская область',
'Bологодская область':'Вологодская область',
'Bоронежская область':'Воронежская область',
'г.Москва':'Москва',
'Еврейская авт. область':'Еврейская автономная область',
'Ненецкий авт. округ':'Ненецкий АО.(Арх.обл.)',
'Республика Саха(Якутия)':'Республика Саха (Якутия)',
'Ханты-Мансийский авт. округ':'ХМАО(Тюменская обл.)',
'Чукотский авт. округ':'Чукотский автономный округ',
'Ямало-Ненецкий авт. округ':'ЯНАО(Тюменская обл.)'})

In [40]:
# 14.Характеристика жилищных условий домохозяйств, % от всех домохозяйств

#data_housing_2020_cond = pd.read_excel('./social_russia_data/housing_2020.xlsx',sheet_name='housing_cond',usecols=[0]+list(range(6,9)))

data_housing_2020_cond = pd.read_excel('./social_russia_data/housing_2020.xlsx',sheet_name='housing_cond',usecols=[0]+list(range(2,9)))

data_housing_2020_cond.rename(columns={'Регион':'region'},inplace=True)

# Убираем пробелы и переносы
data_housing_2020_cond['region'] = data_housing_2020_cond['region'].apply(lambda x: x.strip().replace('\n',' '))

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_housing_2020_cond['region'] = data_housing_2020_cond['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

data_housing_2020_cond

,region,"в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство
0,Российская Федерация,80.7,15.4,3.8,0.1,24.2,16.9,2.4
1,Центральный федеральный округ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Белгородская область,86.3,11.7,2.0,0.0,25.1,17.8,2.6
3,Брянская область,94.7,4.6,0.6,0.0,30.5,24.1,3.2
4,Владимирская область,81.8,15.8,2.5,0.0,24.2,16.1,2.2
...,...,...,...,...,...,...,...,...
89,Амурская область,77.3,16.8,5.9,0.0,23.2,17.3,2.3
90,Магаданская область,80.3,15.7,4.1,0.0,23.7,13.8,1.9
91,Сахалинская область,84.3,13.2,2.5,0.0,22.5,17.2,2.1
92,Еврейская автономная область,82.2,13.1,4.6,0.0,20.1,13.8,2.2


In [41]:
# Добавляем данные в наш датасет.
data = data.merge(right=data_housing_2020_cond,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,реальные доходы,номинальная зарплата 2020,начисленная зп. реал. 2020,"в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,100.2,100070.0,103.1,75.6,18.9,5.3,0.2,21.4,13.8,2.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,97.8,58066.0,101.8,77.8,17.3,4.4,0.5,22.3,14.8,2.1
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,100.4,38499.0,103.2,78.4,17.9,3.7,0.0,26.9,18.1,2.8
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,101.1,68667.0,101.0,82.0,14.2,3.7,0.1,25.1,15.7,2.1
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,92.5,43256.0,102.1,81.1,15.2,3.7,0.1,22.8,15.8,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,104.5,36269.0,105.5,86.5,9.3,4.2,0.0,19.5,14.4,2.4
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,99.2,46237.0,103.7,82.2,13.1,4.6,0.0,20.1,13.8,2.2
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,104.3,102843.0,105.0,80.3,15.7,4.1,0.0,23.7,13.8,1.9
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,105.2,120641.0,110.5,79.1,17.7,3.2,0.0,22.5,14.1,1.7


In [44]:
# 15.валовой региональный продукт на душу населения, в рублях.

data_gross_regional_product_2020 = pd.read_excel('./social_russia_data/gross_regional_product_1996_2020.xls',
                                                 header=2,
                                                 usecols=[0,25],
                                                 names=['region','Валовый продукт'])

# Удаляем Тюменскую и Архангельскую области, т.к. есть в данных разбитие по отдельности от АО
data_gross_regional_product_2020 = data_gross_regional_product_2020.drop([23,70]).reset_index(drop=True)

# Убираем пробелы по краям названий регионов
data_gross_regional_product_2020['region'] = data_gross_regional_product_2020['region'].apply(lambda x: x.strip())

# Приведем наши названия регионов к общему виду, который мы используем в основном датасете
data_gross_regional_product_2020['region'] = data_gross_regional_product_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

data_gross_regional_product_2020.head()


,region,Валовый продукт
0,Российская Федерация,640519.0
1,Центральный федеральный округ,854978.5
2,Белгородская область,646569.0
3,Брянская область,347204.5
4,Владимирская область,410443.6


In [45]:
# Добавляем данные в наш датасет.
data = data.merge(right=data_gross_regional_product_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,номинальная зарплата 2020,начисленная зп. реал. 2020,"в том числе домохозяйства, указавшие, что при проживании не испытывают стесненности","в том числе домохозяйства, указавшие, что при проживании испытывают определенную стесненность","в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность",затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство,Валовый продукт
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,100070.0,103.1,75.6,18.9,5.3,0.2,21.4,13.8,2.0,1567644.8
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,58066.0,101.8,77.8,17.3,4.4,0.5,22.3,14.8,2.1,683845.1
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,38499.0,103.2,78.4,17.9,3.7,0.0,26.9,18.1,2.8,460720.2
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,68667.0,101.0,82.0,14.2,3.7,0.1,25.1,15.7,2.1,971158.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,43256.0,102.1,81.1,15.2,3.7,0.1,22.8,15.8,2.2,588270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,36269.0,105.5,86.5,9.3,4.2,0.0,19.5,14.4,2.4,283451.4
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,46237.0,103.7,82.2,13.1,4.6,0.0,20.1,13.8,2.2,400340.5
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,102843.0,105.0,80.3,15.7,4.1,0.0,23.7,13.8,1.9,2035007.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,120641.0,110.5,79.1,17.7,3.2,0.0,22.5,14.1,1.7,2404271.2


In [46]:
# объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, 
# по видам деятельности за 2017–2020 гг. (в тысячах рублей, значение показателя за год, полный круг)

data_regional_production_2020 = pd.read_csv('./social_russia_data/regional_production_2017_2020.csv',
                                                 header=138,
                                                 usecols=[0,1,5],
                                                 names=['region','тип производства','объем отгруженных товаров'])

# удаляем строки, которые связаны с Архангельской и Тюменской областями
data_regional_production_2020 = data_regional_production_2020.drop([105,106,107,108,109,326,327,328,329,330]).reset_index(drop=True)

# убираем пробелы в названиях регионов
data_regional_production_2020['region'] = data_regional_production_2020['region'].apply(lambda x:x.strip())

# приводим к общему виду названия регионов
data_regional_production_2020['region'] = data_regional_production_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

# сводная таблица 
data_regional_production_2020 = data_regional_production_2020.pivot_table(values='объем отгруженных товаров',
                                                                          index='region',
                                                                          columns='тип производства',
                                                                          aggfunc='sum').reset_index()
data_regional_production_2020.head()


тип производства,region,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ",ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА",ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,Промышленное производство (промышленность)
0,Алтайский край,10454410.50,9146146.1,44741016.30,3.618801e+08,4.262217e+08
1,Амурская область,3471369.50,116722688.8,43000318.70,3.909386e+07,2.022882e+08
2,Архангельская обл.(кроме НАО),5852056.12,30698169.7,36359400.42,2.591439e+08,3.320535e+08
3,Астраханская область,6497362.30,249844173.9,29929257.80,7.602808e+07,3.622989e+08
4,Белгородская область,11175025.30,189803813.7,26787381.90,7.463807e+08,9.741469e+08


In [47]:
# добавляем данные в основной датасет
data = data.merge(right=data_regional_production_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,затруднились ответить,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство,Валовый продукт,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ",ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА",ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,Промышленное производство (промышленность)
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,0.2,21.4,13.8,2.0,1567644.8,1.951764e+08,1.092664e+09,9.642624e+08,7.529105e+09,9.781207e+09
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,0.5,22.3,14.8,2.1,683845.1,1.142418e+08,1.384041e+07,2.786495e+08,3.116646e+09,3.523378e+09
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,0.0,26.9,18.1,2.8,460720.2,3.052297e+07,2.433083e+07,1.134998e+08,9.189983e+08,1.087352e+09
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,0.1,25.1,15.7,2.1,971158.0,7.137330e+07,3.589763e+07,2.036175e+08,2.564838e+09,2.875727e+09
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,0.1,22.8,15.8,2.2,588270.0,7.760801e+07,9.252677e+07,2.451757e+08,2.072197e+09,2.487507e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,0.0,19.5,14.4,2.4,283451.4,5.861507e+05,8.514502e+05,2.739793e+06,4.534270e+06,8.711664e+06
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,0.0,20.1,13.8,2.2,400340.5,4.477501e+05,1.709371e+07,4.967490e+06,5.460453e+06,2.796941e+07
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,0.0,23.7,13.8,1.9,2035007.0,1.134120e+06,3.863604e+07,1.815089e+07,2.098542e+08,2.677753e+08
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,0.0,22.5,14.1,1.7,2404271.2,6.473567e+05,2.313692e+07,1.362003e+07,9.323680e+07,1.306411e+08


In [49]:
# оборот розничной торговли на душу населения, в рублях.
data_retail_turnover_per_capita_2020 = pd.read_excel('./social_russia_data/retail_turnover_per_capita_2000_2021.xls',
                                                     header=3,
                                                     usecols=[0,22],
                                                     names=['region','оборот розничной торговли на душу населения'])

# Удаляем строки, которые являются суммами по регионам Тюменская область и Архангельская
data_retail_turnover_per_capita_2020 = data_retail_turnover_per_capita_2020.drop(index=[23,25,71,74])

# убираем пропуски в названиях регионов
data_retail_turnover_per_capita_2020['region'] = data_retail_turnover_per_capita_2020['region'].apply(lambda x:x.strip())

# приводим к общему виду названия регионов
data_retail_turnover_per_capita_2020['region'] = data_retail_turnover_per_capita_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

data_retail_turnover_per_capita_2020.head()

,region,оборот розничной торговли на душу населения
0,Российская Федерация,231283.0
1,Центральный федеральный округ,301192.0
2,Белгородская область,237076.0
3,Брянская область,225643.0
4,Владимирская область,177680.0


In [50]:
# добавляем данные в наш датасет
data = data.merge(right=data_retail_turnover_per_capita_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Размер общей площади в расчете на члена домохозяйства,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство,Валовый продукт,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ",ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА",ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,Промышленное производство (промышленность),оборот розничной торговли на душу населения
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,21.4,13.8,2.0,1567644.8,1.951764e+08,1.092664e+09,9.642624e+08,7.529105e+09,9.781207e+09,408674.0
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,22.3,14.8,2.1,683845.1,1.142418e+08,1.384041e+07,2.786495e+08,3.116646e+09,3.523378e+09,347566.0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,26.9,18.1,2.8,460720.2,3.052297e+07,2.433083e+07,1.134998e+08,9.189983e+08,1.087352e+09,261857.0
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,25.1,15.7,2.1,971158.0,7.137330e+07,3.589763e+07,2.036175e+08,2.564838e+09,2.875727e+09,283494.0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,22.8,15.8,2.2,588270.0,7.760801e+07,9.252677e+07,2.451757e+08,2.072197e+09,2.487507e+09,259875.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,19.5,14.4,2.4,283451.4,5.861507e+05,8.514502e+05,2.739793e+06,4.534270e+06,8.711664e+06,135351.0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,20.1,13.8,2.2,400340.5,4.477501e+05,1.709371e+07,4.967490e+06,5.460453e+06,2.796941e+07,168066.0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,23.7,13.8,1.9,2035007.0,1.134120e+06,3.863604e+07,1.815089e+07,2.098542e+08,2.677753e+08,245817.0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,22.5,14.1,1.7,2404271.2,6.473567e+05,2.313692e+07,1.362003e+07,9.323680e+07,1.306411e+08,217089.0


In [51]:
# дополнительно дабавим новые названия регионов в наш словарь
dct_city.update({'город Санкт - Петербург':'Санкт-Петербург','Тюменская область без автономного округа':'Тюменская обл.(кроме ХМАО,ЯНАО)',
                 'Ханты-Мансийский АО':'ХМАО(Тюменская обл.)','Ямало-Hенецкий АО':'ЯНАО(Тюменская обл.)'})

In [52]:
# сведения о заболеваемости алкоголизмом на 100 тыс. населения (2005–2018)

data_alco_2018 = pd.read_excel(io='./social_russia_data/drug_alco.xlsx',
                               sheet_name='alco1718',
                               usecols=[0,2],
                               names=['region','алкоголизм 2018'])


# приводим к общему виду названия регионов
data_alco_2018['region'] = data_alco_2018['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

data_alco_2018.head()

,region,алкоголизм 2018
0,Российская Федерация,51.6
1,Центральный федеральный округ,44.9
2,Белгородская область,41.3
3,Брянская область,82.2
4,Владимирская область,66.5


In [53]:
# так же добавляем данные в наш датасет
data = data.merge(right=data_alco_2018,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Размер жилой площади в расчете на члена домохозяйства,Число жилых комнат в расчете на одно домохозяйство,Валовый продукт,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ",ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА",ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,Промышленное производство (промышленность),оборот розничной торговли на душу населения,алкоголизм 2018
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,13.8,2.0,1567644.8,1.951764e+08,1.092664e+09,9.642624e+08,7.529105e+09,9.781207e+09,408674.0,23.1
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,14.8,2.1,683845.1,1.142418e+08,1.384041e+07,2.786495e+08,3.116646e+09,3.523378e+09,347566.0,37.7
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,18.1,2.8,460720.2,3.052297e+07,2.433083e+07,1.134998e+08,9.189983e+08,1.087352e+09,261857.0,18.3
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,15.7,2.1,971158.0,7.137330e+07,3.589763e+07,2.036175e+08,2.564838e+09,2.875727e+09,283494.0,14.7
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,15.8,2.2,588270.0,7.760801e+07,9.252677e+07,2.451757e+08,2.072197e+09,2.487507e+09,259875.0,50.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,14.4,2.4,283451.4,5.861507e+05,8.514502e+05,2.739793e+06,4.534270e+06,8.711664e+06,135351.0,71.1
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,13.8,2.2,400340.5,4.477501e+05,1.709371e+07,4.967490e+06,5.460453e+06,2.796941e+07,168066.0,87
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,13.8,1.9,2035007.0,1.134120e+06,3.863604e+07,1.815089e+07,2.098542e+08,2.677753e+08,245817.0,127
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,14.1,1.7,2404271.2,6.473567e+05,2.313692e+07,1.362003e+07,9.323680e+07,1.306411e+08,217089.0,208.7


In [54]:
# сведения о заболеваемости наркоманией на 100 тыс. населения 2018

data_drug_2018 = pd.read_excel(io='./social_russia_data/drug_alco.xlsx',
                               sheet_name='drug1718',
                               usecols=[0,2],
                               names=['region',' наркомания 2018'])


data_drug_2018['region'] = data_drug_2018['region'].apply(lambda x: dct_city[x] if x in dct_city else x)


data_drug_2018.head()

,region,наркомания 2018
0,Российская Федерация,10.1
1,Центральный федеральный округ,9.2
2,Белгородская область,3.2
3,Брянская область,6.1
4,Владимирская область,7.8


In [55]:
# добавляем в датасет
data = data.merge(right=data_drug_2018,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,Число жилых комнат в расчете на одно домохозяйство,Валовый продукт,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОРА И УТИЛИЗАЦИИ ОТХОДОВ, ДЕЯТЕЛЬНОСТЬ ПО ЛИКВИДАЦИИ ЗАГРЯЗНЕНИЙ",ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПАРОМ; КОНДИЦИОНИРОВАНИЕ ВОЗДУХА",ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,Промышленное производство (промышленность),оборот розничной торговли на душу населения,алкоголизм 2018,наркомания 2018
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,2.0,1567644.8,1.951764e+08,1.092664e+09,9.642624e+08,7.529105e+09,9.781207e+09,408674.0,23.1,8.7
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,2.1,683845.1,1.142418e+08,1.384041e+07,2.786495e+08,3.116646e+09,3.523378e+09,347566.0,37.7,11
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,2.8,460720.2,3.052297e+07,2.433083e+07,1.134998e+08,9.189983e+08,1.087352e+09,261857.0,18.3,3.9
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,2.1,971158.0,7.137330e+07,3.589763e+07,2.036175e+08,2.564838e+09,2.875727e+09,283494.0,14.7,9.4
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,2.2,588270.0,7.760801e+07,9.252677e+07,2.451757e+08,2.072197e+09,2.487507e+09,259875.0,50.2,12.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,2.4,283451.4,5.861507e+05,8.514502e+05,2.739793e+06,4.534270e+06,8.711664e+06,135351.0,71.1,9.2
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,2.2,400340.5,4.477501e+05,1.709371e+07,4.967490e+06,5.460453e+06,2.796941e+07,168066.0,87,65.4
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,1.9,2035007.0,1.134120e+06,3.863604e+07,1.815089e+07,2.098542e+08,2.677753e+08,245817.0,127,6.2
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,1.7,2404271.2,6.473567e+05,2.313692e+07,1.362003e+07,9.323680e+07,1.306411e+08,217089.0,208.7,0


In [56]:
dct_city.update({'Кемеровская область – Кузбасс':'Кемеровская область','Новгородская  область':'Новгородская область',
                 'Республика Северная Осетия – Алания':'Республика Северная Осетия-Алания',
                 'Ханты-Мансийский автономный округ –  Югра':'ХМАО(Тюменская обл.)'})

In [57]:
data_crimes_2020 = pd.read_excel(io='./social_russia_data/crimes2020.xls',header=11,
                                 usecols=[1,2,3,4,5,6,7,8,9,10,11],
                                 names=['region','несовершеннолетними или при их соучастии','ранее совершавшими преступления',
                            'в том числе ранее судимыми','группой лиц','группой лиц по предварительному сговору',
                            'организованной группой','преступным сообществом (преступной организацией)',
                            'в состоянии алкогольного опьянения','наркотического опьянения','токсического опьянения'])

# приведем к общему виду названия регионов
data_crimes_2020['region'] = data_crimes_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
data_crimes_2020.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,region,несовершеннолетними или при их соучастии,ранее совершавшими преступления,в том числе ранее судимыми,группой лиц,группой лиц по предварительному сговору,организованной группой,преступным сообществом (преступной организацией),в состоянии алкогольного опьянения,наркотического опьянения,токсического опьянения
0,Российская Федерация,37771,617184,344383,3456,71460,15382,2345,315440,8522,100
1,Центральный федеральный округ,5589,111086,65904,667,16454,6586,673,55199,2370,14
2,Белгородская область,219,4599,2787,15,505,26,129,2251,49,0
3,Брянская область,197,5177,2994,26,539,125,1,3094,18,3
4,Владимирская область,378,5546,3381,16,602,33,7,3078,21,1


In [58]:
# дабавляем в датасет данные
data = data.merge(right=data_crimes_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,...,несовершеннолетними или при их соучастии,ранее совершавшими преступления,в том числе ранее судимыми,группой лиц,группой лиц по предварительному сговору,организованной группой,преступным сообществом (преступной организацией),в состоянии алкогольного опьянения,наркотического опьянения,токсического опьянения
0,Москва,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,64527.0,...,480,16228,9037,103,4090,4599,151,6614,891,5
1,Московская область,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,35526.0,...,767,21162,14852,262,3455,481,17,9491,714,0
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,40908.0,...,936,18776,10268,97,2523,149,7,6290,119,3
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,28592.0,...,443,10395,6062,71,1484,375,20,3099,457,0
4,Свердловская область,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,27746.0,...,1613,20390,12233,90,2547,233,4,11736,146,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,2134.0,...,125,2326,1123,1,123,10,0,1529,2,0
81,Еврейская автономная область,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,1369.0,...,66,1044,577,4,50,3,0,509,5,0
82,Магаданская область,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,700.0,...,59,932,512,12,43,24,12,592,7,0
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,283.0,...,15,322,143,3,11,0,0,282,0,0


Итак у нас есть готовый датасет состоящий из 67 признаков и 85 наблюдений. 

### 1.2 Описание данных

##### 1.2.1 Численность населения

In [59]:
# Численность населения по годам, расчитывается на 1 января каждого года
data_population = pd.read_excel('./social_russia_data/population.xlsx')
data_population = data_population.iloc[2:,:].drop(['Unnamed: 1'],axis=1)
col = ['Регион']
col +=[x for x in range(1999,2023,1)]

data_population.columns = col
data_population['Регион'] = data_population['Регион'].shift(1) 
mask = data_population[data_population['Регион']=='Российская Федерация'].T.reset_index()
mask = mask.iloc[1:,:].rename(columns={'index':'Год',3:'Численность в млн.'})


data_fig = [go.Scatter(x=mask['Год'],
                 y=mask['Численность в млн.'],
                 name='Популяция'),
                 go.Scatter(x=mask[mask['Год']==2020]['Год'],y=mask[mask['Год']==2020]['Численность в млн.'],
                         name=mask[mask['Год']==2020]['Численность в млн.'].iloc[0],
                         marker=dict(color='Green',
                         size=10))] 

data_layout = go.Layout(title='Численность России в период с 1999 по 2021',
                        legend=dict(x=5),
                        xaxis=dict(title='Год'),
                        yaxis=dict(title='Млн.человек'))

fig = go.Figure(data=data_fig,
                layout=data_layout)

fig.show()

Видим, что численность населения страны падало до 143 млн. человек, а после резко возрастало буквально за один год. Резкий рост связан с присоединением новых регионов. Для анализа будем использовать 2018-2021 годы. Но численность населения возьмем за 2020 год.

Далее в наш датасет добавим метку, которая будет показывать миллионник ли регион или нет. А так же сделаем словарь с цветом каждого региона.

In [60]:
colors = [ '#1F77B4', # Azure Blue
    '#FF7F0E', # Tangerine
    '#2CA02C', # Forest Green
    '#D62728', # Ruby Red
    '#9467BD', # Violet Iris
    '#8C564B', # Sandstone
    '#E377C2', # Coral Blush
    '#BCBD22'  # Lime Yellow
]
dct_colors = dict()
for num,value in enumerate(data['Федеральный округ'].unique()):
    dct_colors[value]=colors[num]

# Метка для определения уровня численности населения на графике.
label_region = data['Численность населения'].apply(lambda x: np.floor(x/1_000_000)) 
data.insert(1,'уровень населения',label_region)

data

,Регионы,уровень населения,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,...,несовершеннолетними или при их соучастии,ранее совершавшими преступления,в том числе ранее судимыми,группой лиц,группой лиц по предварительному сговору,организованной группой,преступным сообществом (преступной организацией),в состоянии алкогольного опьянения,наркотического опьянения,токсического опьянения
0,Москва,12.0,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,...,480,16228,9037,103,4090,4599,151,6614,891,5
1,Московская область,7.0,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,...,767,21162,14852,262,3455,481,17,9491,714,0
2,Краснодарский край,5.0,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,...,936,18776,10268,97,2523,149,7,6290,119,3
3,Санкт-Петербург,5.0,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,...,443,10395,6062,71,1484,375,20,3099,457,0
4,Свердловская область,4.0,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,...,1613,20390,12233,90,2547,233,4,11736,146,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,0.0,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,...,125,2326,1123,1,123,10,0,1529,2,0
81,Еврейская автономная область,0.0,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,...,66,1044,577,4,50,3,0,509,5,0
82,Магаданская область,0.0,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,...,59,932,512,12,43,24,12,592,7,0
83,Чукотский автономный округ,0.0,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,...,15,322,143,3,11,0,0,282,0,0


In [61]:
# Создаем маску, где будем использовать цвета наших регионов.
mask_number = data.groupby('Федеральный округ')['Численность населения'].sum().reset_index()
mask_number.sort_values(by='Численность населения',ascending=False,inplace=True)
mask_number['цвет'] = mask_number['Федеральный округ'].map(dct_colors)

# Нарисуем первую фигуру, которая будет состоять из гистограммы и коробчатой диаграммы
fig_1=make_subplots(rows=1,
                  cols=2,
                  subplot_titles=['Гистограмма ','Коробчатая диаграмма']
                  )
# Гистограмма
fig_1.add_trace(col=1,row=1,trace=go.Histogram(x=data['Численность населения'],showlegend=False))
fig_1.update_xaxes(title='Численность',col=1,row=1)

# Коробчатая диаграмма
fig_1.add_trace(col=2,row=1,trace=go.Box(x=data['Численность населения'],showlegend=False,name=''))
fig_1.update_xaxes(title='Численность',col=2,row=1)
# Изменим высоту и ширину первого графика
fig_1.update_layout(height=260,width=1200)

# Нарисуем первый график
fig_1.show()

# Далее нам понадобится столбчатая диаграмма, цветом выделим округа, а меткой сделаем регионы-млн.
fig_2 = px.bar(x=data['Регионы'],
             title='Численность населения по регионам',
             y=data['Численность населения'],
             color=data['Федеральный округ'],
             color_discrete_map=dct_colors,
             height=700,width=1500)

# Отсортируем согласно признаку Регионы, а он у нас отсортирован по численности населения
fig_2.update_xaxes(categoryorder='array',categoryarray=data['Регионы'])

# Добавим текст во внутрь черным цветом, что бы мы понимали где регион миллионник
fig_2.update_traces(textposition='outside',textfont_color='#3F51B5')


# Третий график будет внутри основного второго графика
fig_2.add_trace(
    trace=go.Bar(marker_color=mask_number['цвет'],
        x=mask_number['Федеральный округ'],
        y=mask_number['Численность населения'],
        name="Федеральный округ",xaxis='x2',
        showlegend=False))

# Рисуем внутри второго графика фигуру типа rect - это прямоугольник, указываем его вершины
fig_2.update_layout( shapes=[
    dict(type="rect", #тип
         xref="paper", # цвет 
         yref="paper", # цвет 
         x0=0.5, # начало графика x0 от левого края
         y0=0.35, # начало графика y0 от нижнего края
         x1=0.9, # конец графика x1 от левого края
         y1=0.83, # конец графика y1 от нижнего края
         line_width=2, #
         fillcolor=None, #
         opacity=1)])

# Уточняем область отображения второго графика
fig_2.update_traces(selector=lambda t: t.name=="Федеральный округ", xaxis="x2", yaxis="y2")

# Новые оси третьего графика
fig_2.update_layout(
    xaxis2=dict(domain=[0.51, 0.88], anchor="y2"),  # определяем оси X для врезанного графика
    yaxis2=dict(domain=[0.37, 0.81], anchor="x2")) # определяем оси Y для врезанного графика

fig_2.update_layout(autosize=False,
                  legend_orientation="h",
                  legend=dict(x=0,y=1.2, yanchor='top',title=''),
                  title={'text':"Численность населения",'x':0.4,'y':1},
                  xaxis_title="Регионы",
                  yaxis_title="Численность",
                  
                  xaxis2 = {'showticklabels': False},
                  yaxis2 = {'showticklabels': False})



fig_2.show()

По данному признаку можно увидеть, что:

1) Есть выбросы такие как Москва-12млн., МО-7.6 млн.,Краснодарский край-5.6 млн. и Санкт-Петербург-5.4млн. человек. Так и есть в    действительности это самые густонаселенные регионы нашей страны.
2) Центральный федеральный округ самый крупный, т.к. там есть Москва и МО., при этом самый маленький это Дальневосточный федеральный округ.
3) Самый большой регион Москва, самый маленький регион по численности это Ненецкий АО. 

##### 1.2.2 Новорожденные

In [ ]:
# Создаем маску для формирования нужного нам датасета.
mask_newborn = data.sort_values(by='Новорожденные',ascending=False)
mask_newborn['ОКР'] = mask_newborn['Новорожденные']*100/mask_newborn['Численность населения']
mask_newborn_OKR = mask_newborn.sort_values(by='ОКР',ascending=False)

mask_perc = data.groupby('Федеральный округ')[['Численность населения','Новорожденные']].sum().reset_index()
mask_perc['Новорожденные в %'] = mask_perc['Новорожденные']*100/mask_perc['Численность населения'].astype('float')
mask_perc.sort_values(by='Новорожденные в %',ascending=False,inplace=True)
mask_perc['Новорожденные в %'] = mask_perc['Новорожденные в %'].round(2)
mask_perc['цвет'] = mask_perc['Федеральный округ'].map(dct_colors)


fig = make_subplots(rows=2,
                    cols=1,
                    vertical_spacing=0,
                    shared_xaxes=True,row_titles=['%','кол-во'])


for group in mask_newborn_OKR['Федеральный округ'].unique():
    mask = mask_newborn_OKR[mask_newborn_OKR['Федеральный округ']==group]
    fig.add_trace(col=1,
                  row=1,
                  trace=go.Bar(x=mask['Регионы'],
                  y=mask['ОКР'],
                  showlegend=True,
                  name=group,
                  text=mask['уровень населения'],
                  textfont={'size': 10},
                  legend='legend1',
                  textposition='outside',
                  textfont_color='#3F51B5', 
                  marker_color = dct_colors[group]))

fig.add_trace(go.Bar(x=mask_newborn_OKR['Регионы'],
                     y=mask_newborn_OKR['Новорожденные'],
                     marker_color='blue',name='НОВОРОЖДЕННЫЕ'),row=2,col=1)                  
    
fig.update_xaxes(categoryorder='array',categoryarray=mask_newborn_OKR['Регионы'],row=1,col=1) 
fig.update_yaxes(range=[0, 3],row=1,col=1)  
fig.update_yaxes(autorange="reversed", row=2, col=1)

fig.update_layout(height=700,
                  barmode='relative',
                  title=dict(text='Новорожденные',x=0.4,y=1),
                  width=1500,
                  legend=dict(x=0, y=1.25,title='Федеральный округ'), legend_orientation="h")


# Добавляем вставленный график
fig.add_trace(go.Box(x=data['Новорожденные'],name="",showlegend=False,xaxis="x3",yaxis="y3"))


# Конфигурация осей для вторичного графика

fig.update_layout(shapes=[dict(type="rect", #тип
                           xref="paper", # цвет 
                           yref="paper", # цвет 
                           x0=0.6, # начало графика x0 от левого края
                           y0=0.1, # начало графика y0 от нижнего края
                           x1=0.95, # конец графика x1 от левого края
                           y1=0.3, # конец графика y1 от нижнего края
                           line_width=2, #
                           fillcolor=None, #
                           opacity=1)], # прозрачность
)

# # Уточняем область отображения второго графика


fig.update_layout(
     xaxis3=dict(domain=[0.6, 0.95], anchor="y3"),  # определяем оси X для врезанного графика
     yaxis3=dict(domain=[0.1, 0.3], anchor="x3"))   # определяем оси Y для врезанного графика


fig.update_layout(
    xaxis3_title="",  # Убираем подпись оси X
    yaxis3_title=""   # Убираем подпись оси Y
)


fig.show()

In [63]:
fig_1 = px.bar(color=mask_perc['Федеральный округ'],
               color_discrete_map=dct_colors,
               x=mask_perc['Федеральный округ'],
               y=mask_perc['Новорожденные в %'],
               width=1500,
               height=500)
fig_1.update_layout(legend=dict(title='Федеральный округ'),
                    title=dict(text='Прирост населения по федеральным округам в процентах',x=0.4,y=1))

fig_1.show()  

На графике четко видно, что регионы с большой численностью населения в чистом виде конечно дают больше прирост, но если брать в процентном, то общая картина такова, что не более 2.1% прирост населения. И то данный показатель виден только в Чеченской республике и республика Тыва, у которой численность населения составляет 328 тыс. человек. Ну и брать если округа, то тут на первом месте мы видим Северо-Кавказкий округ, далее идет Дальневосточный, Уральский, Сибирский. Ну а потом идет центральная Россия. На Кавказе традиционно высокая рождаемость, т.к. это восточное население со своими устоями, детей в семье не меньше 3-4 и более. А вот Дальневосточный, Уральский и Сибирский округа показывают рост из-за высокой поддержки государства.

##### 1.2.3 число умерших детей на первом году жизни (город, деревня)

In [64]:
fig=make_subplots(rows=1,
                  cols=2)
fig.add_box(x=data['число умерших детей на первом году жизни в городе 2020'],row=1,col=1,name='город')
fig.add_box(x=data['число умерших детей на первом году жизни в деревне 2020'],row=1,col=2,name='деревня')

fig.update_layout(title={'text':'Смертность на первом году жизни','x':0.5},
                  height=300, # общаая высота графика
                  width=1500,
                  )
fig.show()

Видим, что так же присутствуют выбросы как и в численности населения, т.к. чем больше неаселение тем больше рождаемость и количество смертей.
Для понимания проблемы со смертностью, необходимо рассмотреть цифры на уровне региона и федеральных округов. 

In [65]:
mask = data.copy()
mask = mask.groupby('Федеральный округ')[['Новорожденные','число умерших детей на первом году жизни в деревне 2020','число умерших детей на первом году жизни в городе 2020']].sum()
mask['% деревня'] = mask['число умерших детей на первом году жизни в деревне 2020']*100/mask['Новорожденные']
mask['% город'] = mask['число умерших детей на первом году жизни в городе 2020']*100/mask['Новорожденные']
mask['sum'] = mask['% деревня'] + mask['% город']
mask_sort = mask.sort_values(by='sum',ascending=False)
mask_sort = mask_sort.reset_index()


fig = go.Figure(layout={'title':'Смертность на первом году жизни','height':350})
fig.add_trace(go.Bar(x=mask_sort['Федеральный округ'],y=mask_sort['% город'],name='Город'))
fig.add_trace(go.Bar(x=mask_sort['Федеральный округ'],y=mask_sort['% деревня'],name='Деревня'))

Северо-Кавказкий федеральный округ имеет самые высокие показатели по смертности на первом году жизни, причем показатель высокий по двум местностям, город и деревня.А центральный, несмотря даже на огромную цифру в Москве занимает предпоследнее место.

А если смотреть в разрезе каждого региона, то картина следующая

In [66]:
mask_fed = data.sort_values(by='Федеральный округ',ascending=False,ignore_index=True) 
mask_fed['% умерших детей в деревне'] = ((mask_fed['число умерших детей на первом году жизни в деревне 2020']/mask_fed['Новорожденные']
                                          )*100).astype("float16")

mask_fed['% умерших детей в городе'] = ((mask_fed['число умерших детей на первом году жизни в городе 2020']/mask_fed['Новорожденные']
                                         )*100).astype("float16")       

fig = make_subplots(2,1,shared_xaxes=True,vertical_spacing=0,row_titles=['%','кол-во'])

for group in mask_fed['Федеральный округ'].unique():
    mask = mask_fed[mask_fed['Федеральный округ']==group]
    fig.add_trace(col=1,
                  row=1,
                  trace=go.Bar(x=mask['Регионы'],
                  y=mask['% умерших детей в городе'],
                  showlegend=True,
                  name=group,
                  text=mask['уровень населения'],
                  legend='legend1',
                  textposition='outside',
                  textfont={'size':10},
                  textfont_color='#3F51B5', 
                  marker_color = dct_colors[group]))


fig.add_trace(go.Bar(x=mask_fed['Регионы'],
                     y=mask_fed['число умерших детей на первом году жизни в городе 2020'],
                     marker_color='black'),row=2,col=1) 

fig.update_layout(height=600,title=dict(text='% умерших детей на первом году жизни в городе',x=0.4,y=1),
                  width=1500,
                  legend=dict(x=0, y=1.23,title='Федеральный округ'), legend_orientation="h")
fig.update_yaxes(autorange="reversed", row=2, col=1)
fig.update_yaxes(range=[0,1.5], row=1, col=1)
fig.show()

Ситуация со смертностью на первом году жизни в городе примерно ровная. Есть конечно скачки, как в абсолютном значении Москва и МО, так и в процентном Чукотский автономный округ. На Алтае минимальная смертность. Конечно есть определенная зависимость миллионников и смертностью на первом году жизни именно в городе.

In [67]:
fig = make_subplots(2,1,shared_xaxes=True,vertical_spacing=0,row_titles=['%','кол-во'])

for group in mask_fed['Федеральный округ'].unique():
    mask = mask_fed[mask_fed['Федеральный округ']==group]
    fig.add_trace(col=1,
                  row=1,
                  trace=go.Bar(x=mask['Регионы'],
                  y=mask['% умерших детей в деревне'],
                  showlegend=True,
                  name=group,
                  text=mask['уровень населения'],
                  textfont={'size':10},
                  legend='legend1',
                  textposition='outside',
                  textfont_color='#3F51B5', 
                  marker_color = dct_colors[group]))


fig.add_trace(go.Bar(x=mask_fed['Регионы'],
                     y=mask_fed['число умерших детей на первом году жизни в деревне 2020'],
                     marker_color='black'),row=2,col=1) 

fig.update_layout(height=600,title=dict(text='% умерших детей на первом году жизни в деревне',x=0.4,y=1),
                  width=1500,
                  legend=dict(x=0, y=1.23,title='Федеральный округ'), legend_orientation="h")
fig.update_yaxes(autorange="reversed", row=2, col=1)
fig.update_yaxes(range=[0,0.8], row=1, col=1)
fig.show()

А тут ситуация совсем другая. Заметно, что в Северо-Кавказком федеральном округе очень высокая смертность в деревне. Так же в Приволжском округе, но чуть меньше. Центральный округ стабильно низкий показатель среди всех регионов. Так же выделяются регионы, где показатель равен нулю.

##### 1.2.4 Инвалидность

In [68]:
ind_all = data.copy()
ind_all = ind_all.iloc[:,[0,1,2,3]+list(range(7,13))]

for val in ind_all.columns[4:]:
    ind_all[val+' доля от населения']=(ind_all[val]*100/ind_all['Численность населения']).astype("float16")


fig = make_subplots(2,1,shared_xaxes=True,vertical_spacing=0,row_titles=['%','кол-во'])

fig = make_subplots(rows=2,
                    shared_xaxes=True,
                    cols=1,
                    vertical_spacing=0,
                    row_titles=['%','кол-во'])

pallete = ['black','red','blue','green','yellow']
for num,col in enumerate(reversed(data.columns[8:13])):
    fig.add_trace(go.Bar(y=ind_all[col+' доля от населения'],x=ind_all['Регионы'],
                     name=col,legendgroup="legend"+str(num+1),showlegend=False,marker_color=pallete[num]),row=1,col=1)
    
    fig.add_trace(go.Bar(x=data['Регионы'],y=data[col],text=data['уровень населения'],
                         textfont={'size':10},
                     name='60_инвалид',legendgroup="legend"+str(num+1),marker_color=pallete[num]),row=2,col=1)
  
 


fig.update_yaxes(range=[1_150_000,0], row=2, col=1)


fig.update_layout(height=600,title=dict(text='Количество инвалидов по возрастным группам',x=0.4,y=1),
                  width=1500,
                  legend=dict(x=0, y=1.1,title='ВОЗРАСТ : '), legend_orientation="h",barmode='stack')

fig.update_traces(textposition='outside',textfont_color='#3F51B5',row=2,col=1)
fig.show()

Снизу на графике, где показано количество инвалидов, ситуация похожая с численностью населения, есть моменты, где количество инвалидов в регионе кому за 60 увеличивают общую картину, но примерно картина понятна.

А если смотреть сверху, где все исчесляется в процентах от общего населения, тут уже можно сделать ряд выводов.

* Например, в регионах Северо-Кавказкого ФО наблюдается высокий процент всех категорий инвалидов. 

* В Белгородской области высокий показатель инвалидов кому за 60 как и в Питере.
* А самый низкий показатель в сумме в ЯНАО.

Т.е. в абсолютных числах ситуация не показывает всю картину в регионе, только в процентных отношениях можно понять проблему.

1.2.5 За чертой бедности

In [69]:
# подготовим наш датасет к дальнейшей работе и добавим признаки для визуализации
mask_below_the_poverty_line = data.copy()
mask_below_the_poverty_line['за чертой бедности'] = (mask_below_the_poverty_line['Численность населения']*mask_below_the_poverty_line[
    'за чертой бедности в %']/100).astype("int")


# Создаем главную фигуру

fig = make_subplots(rows=2,cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['%','кол-во'])

for group in mask_below_the_poverty_line['Федеральный округ'].unique():
    mask = mask_below_the_poverty_line[mask_below_the_poverty_line['Федеральный округ']==group]
    fig.add_trace(col=1,
                  row=1,
                  trace=go.Bar(x=mask['Регионы'],
                  y=mask['за чертой бедности в %'],
                  showlegend=True,
                  name=group,
                  text=mask['уровень населения'],
                  textfont={'size':10},
                  legend='legend1',
                  textposition='outside',
                  textfont_color='#3F51B5', 
                  marker_color = dct_colors[group]))

fig.add_trace(go.Bar(x=mask_below_the_poverty_line['Регионы'],
                     y=mask_below_the_poverty_line['за чертой бедности'],
                     marker_color='black',showlegend=False),row=2,col=1)    


fig.update_layout(height=650,title=dict(text='Население за чертой бедности',x=0.4,y=0.99),
                  width=1200,
                  legend=dict(x=0, y=1.2,title='Федеральный округ'), legend_orientation="h")                   

fig.update_yaxes(autorange="reversed", row=2, col=1)

fig.update_xaxes(categoryorder='array',categoryarray=mask_below_the_poverty_line['Регионы'],row=2,col=1) 

fig.show()

В чистом виде конечно наблюдается высокая зависимость от численности населения. А если брать в процентах, то тут ситуация немного другая. Заметен конечно высокий уровень бедности в регионах, где численность населения беньше млн., но есть и такие регионы, которые выделяются, к примеру Чеченская республика, Забайкальский край, Крым,Алтайский край, Иркутская область.

Перед тем как мы увидим ситуацию по группам в данной категории, нам необходимо добавить недостающие данные, по регионам, Чукотский автономный округ и Севастополь Возьмем их с 2018 года. Думаю координально ничего не поменялось.

In [70]:
data_poverty_socdem_2018 = pd.read_excel('./social_russia_data/poverty_socdem_2018.xls',header=2,usecols=[0,2,3,4])

col = ['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']
data_poverty_socdem_2018.columns = col

data_poverty_socdem_2018['region'] = data_poverty_socdem_2018['region'].apply(lambda x: x.strip())
data_poverty_socdem_2018['region'] = data_poverty_socdem_2018['region'].apply(lambda x:dct_city[x] if x in dct_city else x)
mask_regions = data_poverty_socdem_2018[data_poverty_socdem_2018['region'].isin(
    data[data['% <16 за чертой бедности'].isna()]['Регионы'])]


mask_regions

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
38,Севастополь,41.6,7.2,51.1
87,Чукотский автономный округ,51.0,0.0,49.0


In [71]:
# Меняем наши пустые значения на значения 2018 года.
data.iloc[[76,83],[14,15,16]] = mask_regions.iloc[:,[1,2,3]]
data.iloc[[76,83],[14,15,16]]

,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
76,41.6,7.2,51.1
83,51.0,0.0,49.0


In [72]:
mask_share = data.copy()
mask_share['за чертой бедности'] = (mask_share['Численность населения']*mask_share[
    'за чертой бедности в %']/100).astype("int")
# Население по группам за чертой бедности в количестве человек.
mask_share['<16 за чертой бедности'] = (mask_share['% <16 за чертой бедности']*mask_share['за чертой бедности']/100).astype("int")
mask_share['16-60 за чертой бедности'] = (mask_share['% 16-60 за чертой бедности']*mask_share['за чертой бедности']/100).astype("int")
mask_share['>60 за чертой бедности'] = (mask_share['% >60 за чертой бедности']*mask_share['за чертой бедности']/100).astype("int")


mask_share['доля <16 за чертой бедности'] = (mask_share['<16 за чертой бедности']*100/mask_share['Численность населения']).astype("float16")
mask_share['доля 16-60 за чертой бедности'] = (mask_share['16-60 за чертой бедности']*100/mask_share['Численность населения']).astype("float16")
mask_share['доля >60 за чертой бедности'] = (mask_share['>60 за чертой бедности']*100/mask_share['Численность населения']).astype("float16")
mask_share


fig = make_subplots(rows=2,cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['доля','%'])

lst_share = ['% >60 за чертой бедности','% 16-60 за чертой бедности','% <16 за чертой бедности']
colors_share = ['red','yellow','green']

for num,col in enumerate(lst_share):
    fig.add_trace(go.Bar(x=mask_share['Регионы'],
                     y=mask_share[col],
                     legendgroup="legend"+str(num),
                     name=col,
                     marker_color=colors_share[num]),row=2,col=1)

for num,column in enumerate(mask_share.columns[[74,73,72]]):
    trace = go.Bar(x=mask_share['Регионы'],
                   y=mask_share[column],
                   legendgroup="legend" + str(num),
                   showlegend=False,
                   name=column,
                   marker_color=colors_share[num])    
    
    if num == len(mask_share.columns[[74, 73, 72]]) - 1:
        trace.text = mask_share['уровень населения']
        trace.textposition = 'outside'
        trace.textfont = {'size':10}
        

    fig.add_trace(trace, row=1, col=1)    

fig.update_layout(height=650,title=dict(text='Население за чертой бедности',x=0.4,y=0.99),
                  width=1500,
                  barmode='stack',
                  legend=dict(x=0.11, y=1.2), legend_orientation="h")  

fig.update_yaxes(autorange="reversed", row=2, col=1)
fig.update_yaxes(range=[0,35], row=1, col=1)

fig.show()


Самые высокие показатели населения за чертой бедности по отношению к населению региона наблюдается в регионах, где численность менее млн. человек. Но опять же в поле зрения попадает Северо-Кавказкий ФО, Чеченская республика, Так же Забайкальский край. А если мы рассматриваем в разрезе распределение групп исходя из 100%, график снизу, выделяется сильно Мордовия, где очень низкий показатель пенсионеров за чертой бедности, но при этом очень высокий показатель среднего возраста.

##### 1.2.6 Болезни

Насчет болезней, у нас представлены статистические данные, которые показывают количество человек на население 100 тыс. Поэтому можно попробовать их суммировать и посмотреть общую тенденцию

Сделаем график относительно Федеральных округов, что бы понять общую картину по болезням. Так же библиотека Plotly позволяет выбрать каждый вид болезни и уже в крации будем анализировать

In [294]:
mask_illnesses = data.sort_values(by='Федеральный округ')

fig = make_subplots(specs=[[{'t':0.25}]])

for n,col in enumerate(mask_illnesses.columns[17:37]):
    trace = go.Bar(x=mask_illnesses['Регионы'],
                  y=mask_illnesses[col],
                  legend="legend1",
                  marker_color=mask_illnesses['Федеральный округ'].map(dct_colors),
                  marker_line_width=0,
                  name=col)
    
    if n == len(mask_illnesses.columns[17:37]) - 1:
        trace.text = mask_illnesses['уровень населения']
        trace.textposition = 'outside'
        trace.textfont = {'size':10}

    fig.add_trace(trace) 
    
# Дополнительные следы для собственной легенды
for value in data['Федеральный округ'].unique():
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=dct_colors[value]),name=value,legend='legend2'))
   
   
fig.update_layout(height=900,
                  title=dict(text='Болезни',x=0.4,y=1),
                  width=1600,
                  barmode='stack',
                  legend1=dict(x=0, y=1.4,orientation="h"),legend2=dict(title='Федеральный округ',x=0,y=0.9,orientation="h") )


fig.show()

Сразу бросается в глаза, что Северо-Кавказкий Федеральный округ меньше всего болеет по статистике, при этом самый большой процент инвалидов. Ситуация конечно спорная, но статистика - вещь упрямая.

Несколько пунктов можно описать
* Болезни кожи самый низкий показатель в Чеченской республике.Самый высокий в Питере
* Болезни крови самый высокий в Дагестане, это связано с географическим положением и в свое время отсутствием выхода к земле и часто были родственные браки.
* В итоге самые "больные" регионы нашей страны, Чукотка,Ненецкий АО, как не странно Питер, Карелия, Архангельская область, Алтайский край  и ЯНАО

##### 1.2.7 Расходы на социальную политику.

In [101]:
fig = go.Figure(layout=dict(width=1500,height=530,title=dict(text='Расходы на социальную политику в процентах',x=0.3,y=0.9)))

for value in data['Федеральный округ'].unique():
    mask = data[data['Федеральный округ']==value]
    fig.add_trace(go.Bar(x=mask['Регионы'],
                         y=mask['расходы на соц.пол. от бюджета %'],
                         marker_color=dct_colors[value],
                         text=mask['уровень населения'],
                         textposition='outside',
                         textfont={'size':10},
                         name=value))

fig.update_legends(title='Федеральный округ')
fig.update_yaxes(range=[0,42])
fig.show()

Видим, что самый высокий процент от бюджета тратится в Северо-Кавказком Федеральном округе. Далее сможем увидеть зарплаты в регионах и картина будет более ясная. Есть регионы, где уровень затрат на соц. политику минимальный,Чукотка 6.4%, Магаданская область - 11.1%, ЯНАО всего 12.6%, Севастополь - 13.3%, Калининградская область - 12.6%. Москва средний показатель - 17.5%. Но все должны понимать, что их процент далеко не такой как в Чеченской республике.

##### 1.2.8 Доходы(Среднедушевые и реальные) и зарплаты(Номинальные и реальные) населения регионов

#### **Среднедушевой доход — это показатель уровня жизни семьи, рассчитываемый как средняя сумма доходов всех членов семьи за определенный период (обычно 3 или 12 месяцев), деленная на количество этих членов семьи.**

#### **Реальные доходы — это показатель покупательной способности денежных средств, который отражает количество товаров и услуг, которые можно приобрести на полученный доход с учетом инфляции и налогов к прошлому году**

In [ ]:
fig = make_subplots(rows=2,cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['среднедушевой',' реальный в %'])

for value in data['Федеральный округ'].unique():
    mask = data[data['Федеральный округ']==value]
    fig.add_trace(go.Bar(x=mask['Регионы'],
                         y=mask['Среднедушевые доходы 2020'],
                         marker_color=dct_colors[value],
                         text=mask['уровень населения'],
                         textposition='outside',
                         textfont={'size':10},
                         name=value))

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['реальные доходы'],
                     marker_color=data['реальные доходы'].apply(lambda x: 'yellow' if x>=100 else 'black'),
                     showlegend=False),row=2,col=1)

fig.update_legends(title='Федеральный округ')
fig.update_layout(width=1500,height=630,title=dict(text='Доход населения',x=0.3,y=0.9))
fig.update_yaxes(range=[0,100_000],row=1,col=1)
fig.update_yaxes(range=[120,0],row=2,col=1)
fig.show()

Про это я писал ранее, что зарплаты в Северо-Кавказком регионе далеко не самые высокие. И инфляция выше чем повышение доходов. Так же видим, что в регионах, где был невысокий уровень поддержки населения зарплаты выше среднего, где-то даже в два раза. В основном в регионах с суровым климотом и связанные с добычей полезных ископаемых и Москва.

#### **Номинальная начисленная заработная плата — это сумма денежных средств, которую работодатель начисляет сотруднику за проделанную работу до вычета налогов (например, НДФЛ) и других обязательных удержаний (например, алиментов). Она фиксируется в расчетной ведомости и включает в себя оклад, надбавки и премии, но не отражает реальную покупательную способность денег, которая зависит от инфляции**

#### **Реальная зарплата — это покупательная способность, то есть сколько продуктов, одежды, услуг и т.д. можно купить на имеющиеся деньги по сравнению к прошлому году**

In [132]:
fig = make_subplots(rows=2,cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['номинальная',' реальная в %'])

for value in data['Федеральный округ'].unique():
    mask = data[data['Федеральный округ']==value]
    fig.add_trace(go.Bar(x=mask['Регионы'],
                         y=mask['номинальная зарплата 2020'],
                         marker_color=dct_colors[value],
                         text=mask['уровень населения'],
                         textposition='outside',
                         textfont={'size':10},
                         name=value))

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['начисленная зп. реал. 2020'],
                     marker_color=data['начисленная зп. реал. 2020'].apply(lambda x: 'yellow' if x>=100 else 'black'),
                     showlegend=False),row=2,col=1)

fig.update_legends(title='Федеральный округ')
fig.update_layout(width=1500,height=630,title=dict(text='Зарплата населения',x=0.3,y=0.9))
fig.update_yaxes(range=[0,100_000],row=1,col=1)
fig.update_yaxes(range=[120,0],row=2,col=1)
fig.show()

Реальная зарплата повысилась почти во всех регионах нашей страны, кроме Калининградской области и Костромской.

##### 1.2.9 Характеристика жилищных условий домохозяйств, % от всех домохозяйств

In [174]:
fig = go.Figure()

for n,col in enumerate(data.columns[[42,44,43,45]]):
    fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data[col],
                     name=col,
                     #text=data['уровень населения'],
                     marker_line_width=0,))

fig.update_layout(barmode='stack',
                  title=dict(text='Характеристика жилищных условий',x=0.3,y=1),
                  legend=dict(title='категории опроса',x=0,y=1.2,orientation='h'),
                  width=1500,
                  height=600)
fig.update_yaxes(title='%')
fig.show()

Особо много информации данные не дали, есть пару моментов, на которые можно обратить внимание, Северная Осетия,Хакасия испытывают большую стесненность 

Возможно есть зависимость между стесненостью и размером жилой площади?

In [316]:
fig = make_subplots(rows=2,cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['стесненность',' площадь'])

for value in data['Федеральный округ'].unique():
    mask = data[data['Федеральный округ']==value]
    fig.add_trace(go.Bar(x=mask['Регионы'],
                         y=mask['в том числе домохозяйства, указавшие, что при проживании испытывают большую стесненность'],
                         marker_color=dct_colors[value],
                         text=mask['уровень населения'],
                         textposition='outside',
                         textfont={'size':10},
                         name=value))

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['Размер жилой площади в расчете на члена домохозяйства'],
                     marker_color="#3B4940",
                     showlegend=False),row=2,col=1)

fig.update_legends(title='Федеральный округ')
fig.update_layout(width=1500,height=630,title=dict(text='Размер жилой площади и уровень стесненности в регионах',x=0.3,y=0.9))
fig.update_yaxes(range=[0,15],row=1,col=1)
fig.update_yaxes(range=[30,0],row=2,col=1)
fig.show()

Действительно, мы наблюдаем на графике, что чем больше площадь, тем ниже уровень стесненности в регионе. Казалось, что в городах, где численность менее 1 млн. не будет проблемы с площадью, но картина не везде такая.

##### 1.2.10. Валовый продукт

In [204]:
fig = px.bar(x=data['Регионы'],
             y=data['Валовый продукт'],
             color_discrete_map=dct_colors,
             color=data['Федеральный округ'],
             title='Валовый продукт',
             text=data['уровень населения'])

fig.update_legends(title=dict(text='Федеральный округ'))
fig.update_traces(textposition='outside')

fig.update_yaxes(range=[0,5700000],title='руб.')
fig.update_xaxes(title='Регионы')
fig.show()

Тут сразу понятно откуда зарплаты высокие. ВВП высокие, где есть добыча полезных ископаемых

##### 1.2.11 Производство в регионах

In [290]:
fig = go.Figure()
mask_prod = data.sort_values(by='Федеральный округ')

for n,col in enumerate(mask_prod.columns[50:55]):
    fig.add_trace(go.Bar(x=mask_prod['Регионы'],
                     y=mask_prod[col],
                     name=col,
                     marker=dict(color=mask_prod['Федеральный округ'].map(dct_colors),line=dict(color='black'))))

fig.update_layout(barmode='stack',
                  title=dict(text='Производство',x=0.3,y=1),
                  legend=dict(title='тип',x=0,y=1.5,orientation='h'),
                  width=1500,
                  height=600)
fig.update_yaxes(title='руб')
fig.show()

Так же видно, что регионы как, ХМАО, ЯНАО и Москва,МО на общем фоне сильно выделяются. А вот ситуация в Алтае, Ингушетии, Калмыкии, Чеченская республика, Дагестан противоположно другая. Там нулевые производства.

1.2.12 Оборот розничной торговли на душу населения

In [228]:
fig = px.bar(x=data['Регионы'],y=data['оборот розничной торговли на душу населения'],
             color_discrete_map=dct_colors,
             color=data['Федеральный округ'],
             text=data['уровень населения'])

fig.update_layout(title=dict(text='Оборот розничной торговли на душу населения',x=0.3,y=1),
                  legend=dict(title='Федеральный округ',x=0,y=1.3,orientation='h'),
                  width=1500,
                  height=600)
fig.update_yaxes(title='руб')
fig.update_xaxes(title='Регионы')
fig.update_traces(textposition='outside')

fig.show()

Опять заметно, что Северо-Кавказкий федеральный округ сильно страдает и очень низкие показатели по торговле. Москва, МО, Сахалин,Хабаровск прям лидеры в данном показателе.

##### 1.2.13 Алкоголизм и наркомания

Статистика по данным признакам считается как количество на 100 тыс. человек в регионе.

In [256]:
mask_alco = data.sort_values(by='Федеральный округ')

fig = make_subplots(2,1,shared_xaxes=True,vertical_spacing=0,row_titles=['Алкоголизм','Наркомания'])

fig.add_trace(go.Bar(x=mask_alco['Регионы'],
                     marker=dict(color=mask_alco['Федеральный округ'].map(dct_colors),line=dict(color='black')),
                     y=mask_alco['алкоголизм 2018'],
                     showlegend=False),col=1,row=1)

fig.add_trace(go.Bar(x=mask_alco['Регионы'],
                     marker=dict(color=mask_alco['Федеральный округ'].map(dct_colors),line=dict(color='black')),
                     y=mask_alco[' наркомания 2018'],showlegend=False),col=1,row=2)

fig.update_layout(width=1500,title='Зависимости в регионах',
                  height=600)
fig.update_yaxes(range=[70,0],row=2,col=1)

fig.show()

Статистика удручающая, бросается в глаза Еврейская автономная область, откуда там столько наркоманов? Так же Приморский край,Севастополь, Воронежская область. Данные конечно за 2018 год, но все же. Алкоголизм то более привычное дело для России. Самые высокие показатели в регионах с суровыми условиями жизни, Чукотка, НАО, Республика Коми, ну и тут неожиданно попадает Пенза. Что касается регионов, которые не богаты на ресурсы, но при этом высокий показатель рождаемости и низкий показатель болезни, имеется в виду Северо-Кавказкий округ, то тут ситуация очень хорошая, во всех регионах низкие показатели по зависимостям. 

In [261]:
data

,Регионы,уровень населения,Численность населения,Федеральный округ,Новорожденные,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,всего инвалидов,18_30_инвалид,31_40_инвалид,...,несовершеннолетними или при их соучастии,ранее совершавшими преступления,в том числе ранее судимыми,группой лиц,группой лиц по предварительному сговору,организованной группой,преступным сообществом (преступной организацией),в состоянии алкогольного опьянения,наркотического опьянения,токсического опьянения
0,Москва,12.0,12678079,Центральный федеральный округ,123524.0,10.0,440.0,991311.0,28857.0,44436.0,...,480,16228,9037,103,4090,4599,151,6614,891,5
1,Московская область,7.0,7690863,Центральный федеральный округ,79107.0,26.0,220.0,412799.0,15893.0,26311.0,...,767,21162,14852,262,3455,481,17,9491,714,0
2,Краснодарский край,5.0,5675462,Южный федеральный округ (с 29.07.2016),59235.0,91.0,139.0,429194.0,18213.0,28292.0,...,936,18776,10268,97,2523,149,7,6290,119,3
3,Санкт-Петербург,5.0,5398064,Северо-Западный федеральный округ,55439.0,0.0,191.0,562262.0,12651.0,20695.0,...,443,10395,6062,71,1484,375,20,3099,457,0
4,Свердловская область,4.0,4310681,Уральский федеральный округ,43991.0,38.0,144.0,271245.0,12950.0,22756.0,...,1613,20390,12233,90,2547,233,4,11736,146,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,0.0,220181,Сибирский федеральный округ,2955.0,21.0,1.0,21180.0,903.0,1685.0,...,125,2326,1123,1,123,10,0,1529,2,0
81,Еврейская автономная область,0.0,158305,Дальневосточный федеральный округ,1605.0,5.0,6.0,12466.0,677.0,1176.0,...,66,1044,577,4,50,3,0,509,5,0
82,Магаданская область,0.0,140149,Дальневосточный федеральный округ,1314.0,0.0,4.0,5063.0,322.0,596.0,...,59,932,512,12,43,24,12,592,7,0
83,Чукотский автономный округ,0.0,50288,Дальневосточный федеральный округ,537.0,1.0,7.0,1641.0,175.0,225.0,...,15,322,143,3,11,0,0,282,0,0


##### 1.2.14 Криминал.

По данным показателям, что бы все было максимально понятно, необходимо привести к статистическому виду, либо на 100 тыс. человек либо процент от общего населения в регионе, а иначе мы просто увидим зависимость цифр от численности населения.

In [315]:
mask_crimes = data.sort_values(by='Федеральный округ')
for col in mask_crimes.columns[58:]:
    mask_crimes[col] = (mask_crimes[col]*100_000/mask_crimes['Численность населения']).astype("int")

fig = make_subplots(2,1,
                    shared_xaxes=True,
                    vertical_spacing=0,
                    row_titles=['на 100 тыс. чел.','количество'])

for n,col in enumerate(mask_crimes.columns[[58,59,61,62,63,64,65,66,67]]):
    trace = go.Bar(x=mask_crimes['Регионы'],
                  y=mask_crimes[col],
                  legend="legend1",
                  marker=dict(color=mask_crimes['Федеральный округ'].map(dct_colors)),
                  marker_line_width=0,
                  name=col)
    
    if n == len(mask_crimes.columns[[58,59,61,62,63,64,65,66,67]]) - 1:
        trace.text = mask_crimes['уровень населения']
        trace.textposition = 'outside'
        trace.textfont = {'size':10}

    fig.add_trace(trace,col=1,row=1) 
    
# Дополнительные следы для собственной легенды
for value in mask_crimes['Федеральный округ'].unique():
    fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers', marker=dict(color=dct_colors[value]),name=value,legend='legend2'))
   

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data.iloc[:,[58,59,61,62,63,64,65,66,67]].sum(axis=1),marker=dict(color='black'),
                     showlegend=False,name='всего'),row=2,col=1)


fig.update_yaxes(range=[52000,0],row=2,col=1)
fig.update_layout(height=900,
                  title=dict(text='Криминальная обстановка в регионах',x=0.4,y=1),
                  width=1600,
                  barmode='stack',
                  legend1=dict(x=0, y=1.25,orientation="h",title='тип преступления'),
                  legend2=dict(title='Федеральный округ',x=0,y=1.1,orientation="h") )

fig.update_xaxes(categoryorder='array',categoryarray=mask_crimes['Регионы'],row=2) 

fig.show()

Все таки статистические данные будут наиболее точные в разрезе страны, т.к. огромную роль играет количество жителей в регионе.

## 3. Подготовка данных

In [ ]:
data['% прироста населения'] = (data['Новорожденные']*100/data['Численность населения']).astype("float16")
data

In [ ]:
data['% умерших детей в деревне'] = ((data['число умерших детей на первом году жизни в деревне 2020']/data['Новорожденные'])*100).astype("float16")

data['% умерших детей в городе'] = ((data['число умерших детей на первом году жизни в городе 2020']/data['Новорожденные'])*100).astype("float16")

data

In [ ]:
print('процент ивалидов от всего населения страны: ', round(data['всего инвалидов'].sum()*100/data['Численность населения'].sum(),2),'%')

In [ ]:
data['% 18_30_инвалид'] = (data['18_30_инвалид']*100/data['Численность населения']).astype("float16")
data['% 31_40_инвалид'] = (data['31_40_инвалид']*100/data['Численность населения']).astype("float16")
data['% 41_50_инвалид'] = (data['41_50_инвалид']*100/data['Численность населения']).astype("float16")
data['% 51_60_инвалид'] = (data['51_60_инвалид']*100/data['Численность населения']).astype("float16")
data['% 60_инвалид'] = (data['60_инвалид']*100/data['Численность населения']).astype("float16")
data

In [ ]:


# Выделим пропуски
data_poverty_socdem_2021[data_poverty_socdem_2021['% <16 за чертой бедности'].isna()]

Нам необходимо либо удалить данные строки из датасета либо найти данные, возьмем данные за 2018г. Уверен, что картина за 2 года кардинально не изменилась.

In [ ]:
data_poverty_socdem_2018 = pd.read_excel('./social_russia_data/poverty_socdem_2018.xls',header=2,usecols=[0,2,3,4])

col = ['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']
data_poverty_socdem_2018.columns = col

data_poverty_socdem_2018['region'] = data_poverty_socdem_2018['region'].apply(lambda x: x.strip())

mask = data_poverty_socdem_2018[data_poverty_socdem_2018['region'].isin(
    data_poverty_socdem_2021[data_poverty_socdem_2021['% <16 за чертой бедности'].isna()]['region'])]

data_poverty_socdem_2021.iloc[[40,91],[1,2,3]] = mask.iloc[:,[1,2,3]]
mask

Нам необходимо в таблице 2021 года заменить значения 40 и 91 строки.

In [ ]:
data_poverty_socdem_2021.iloc[[40,91],[1,2,3]] = mask.iloc[:,[1,2,3]]

In [ ]:
data_poverty_socdem_2021['region'] = data_poverty_socdem_2021['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_poverty_socdem_2021

In [ ]:
data['доля <16 за чертой бедности'] = (data['за чертой бедности в %']*data['% <16 за чертой бедности']/10000).astype("float")
data['доля 16-60 за чертой бедности'] = (data['за чертой бедности в %']*data['% 16-60 за чертой бедности']/10000).astype("float")
data['доля >60 за чертой бедности'] = (data['за чертой бедности в %']*data['% >60 за чертой бедности']/10000).astype("float")
data

In [ ]:
corr_matrix = dfr.iloc[:,1:].corr()

threshold = 0.8
high_corr_features = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i,j]) > threshold:
            high_corr_features.append((corr_matrix.columns[i], corr_matrix.columns[j]))


dct = dict()

for value1,value2 in high_corr_features:
    if value1 not in dct:
        dct[value1] = 1
    else:
         dct[value1] += 1

    if value2 not in dct:
        dct[value2] = 1
    else:
         dct[value2] += 1

sorted(dct.items(),key=lambda item: item[1],reverse = True)

In [ ]:
data_drop = dfr.drop(['ранее совершавшими преступления',
                      '% 41_50_инвалид',
                      'в том числе ранее судимыми',
                      '% 18_30_инвалид',
                      'номинальная зарплата 2020',
                      'объем собствен. производства',
                      'доля 16-60 за чертой бедности',
                      'Размер жилой площади в расчете на члена домохозяйства',
                      'Валовый продукт',
                      'в состоянии алкогольного опьянения'],axis=1)



corr_matrix = data_drop.corr()

threshold = 0.8
high_corr_features = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i,j]) > threshold:
            high_corr_features.append((corr_matrix.columns[i], corr_matrix.columns[j]))


dct = dict()

for value1,value2 in high_corr_features:
    if value1 not in dct:
        dct[value1] = 1
    else:
         dct[value1] += 1

    if value2 not in dct:
        dct[value2] = 1
    else:
         dct[value2] += 1

sorted(dct.items(),key=lambda item: item[1],reverse = True)

## <center> Кодирование признаков.

Стандартизация данных

In [ ]:
scaler = RobustScaler()
scaler.fit(data_drop)
cols = data_drop.columns
data_scaler = scaler.transform(data_drop)
data_scaler = pd.DataFrame(data_scaler,columns=cols)
data_scaler

In [ ]:
from sklearn.decomposition import PCA
# создаём объект класса PCA
# n_components — задаём количество компонентов для проведения трансформации
pca = PCA(n_components=3, random_state=42)
# обучаем модель на данных X
pca.fit(data_scaler)
# применяем уменьшение размерности к матрице X
pca_data = pca.transform(data_scaler) 
pca_data = pd.DataFrame(pca_data)
pca_data

px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Вычисляем сумму квадратичных отклонений для разных чисел кластеров
inertia = []
for k in range(1, 13):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(data_scaler)
    inertia.append(km.inertia_)

plt.plot(range(1, 13), inertia, marker='o')
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('Сумма квадратичных отклонений')
plt.show()

In [ ]:
from yellowbrick.cluster import KElbowVisualizer  # Используется только для K-MEANS
Elbow_M = KElbowVisualizer(KMeans(random_state=42), k=10) 
Elbow_M.fit(data_scaler)
Elbow_M.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score
sil_scores = []
for k in range(2, 11):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(data_scaler)
    score = davies_bouldin_score(data_scaler, labels)
    sil_scores.append(score)

plt.plot(range(2, 11), sil_scores, marker='o')
plt.title('Davies-Bouldin Index')
plt.xlabel('Количество кластеров')
plt.ylabel('индекс')
plt.show()

In [ ]:
k_means =  KMeans(n_clusters=4,random_state=42)
k_means.fit(data_scaler)

k_means.labels_

In [ ]:
px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400,color=k_means.labels_)

In [ ]:


px.scatter(x=pca_data[0],y=pca_data[1],width=1200,height=400,color=k_means.labels_)

In [ ]:
data['clusters'] = k_means.labels_
data[data['clusters']==0]

## <center> Построение модели

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agglomerative_clustering = AgglomerativeClustering(n_clusters=4)
agglomerative_clustering.fit(data_scaler)
agglomerative_clustering.labels_



In [ ]:

data['clusters'] = agglomerative_clustering.labels_
data[data['clusters']==2]

In [ ]:
from sklearn.cluster import DBSCAN

# Запускаем кластеризацию на наших данных
model_2 = DBSCAN(eps=0.5, min_samples=4)

model_2.fit(data_scaler)

px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400,color=model_2.labels_)
